In [12]:
import numpy as np
import pandas as pd
import os
import time

In [13]:
def data_reader(excelPaths):
    dataset = []
    for key, value in excelPaths.items():
        for path in value[0]:
            print(path)
            
            x = pd.read_excel(path, sheet_name=['Mieterliste', 'Internationales Verfahren DCF'], header=None)
            
            #Time of the last modification in seconds
            ti_m = os.path.getmtime(path)
            #Time of the last modification standart
            m_ti = time.ctime(ti_m)
            
            features = collect_data(x)
            features.insert(0, key)
            features.append(ti_m)
            features.append(m_ti)
            dataset.append(features)
            
        #AMP - average market rent
        #Wohnfläche - LivingArea
        #Gewerbefläche - CommercialArea
        #JNKM  - Annual net cold rent - ANCA
        #LM_Timecode - Last Modified timecode
        #LM_date - Last Modified standart
        #Instandhaltungsrücklage - Maintenance reserve
        #Capex - Capital expenditure
        df = pd.DataFrame(dataset, columns =['City', 'Address', 'LivingArea', 'CommercialArea', 
                                             'ANCA', 'DCF_Price', 'AMP', 'MaintenanceReserve', 'Capex',
                                             'LM_Timecode', 'LM_date']) 
        writeFiles(df, key)
    return dataset
    
#Function to extract data from the Excel file based on the Excel sheet name and cell number        
def collect_data(excelFile):
    infos = []
    
    infos.append(str(excelFile['Internationales Verfahren DCF'].loc[1, 3]))          #Adresse
    infos.append(excelFile['Mieterliste'].loc[26, 8])                                #Wohnfläche
    infos.append(excelFile['Mieterliste'].loc[27, 8])                                #Gewerbefläche
    infos.append(excelFile['Internationales Verfahren DCF'].loc[82, 3])              #JNKM
    infos.append(excelFile['Internationales Verfahren DCF'].loc[12, 8])              #DCF Preis
    infos.append(excelFile['Internationales Verfahren DCF'].loc[26, 8])              #m²-Miete Marktüblich
    infos.append(excelFile['Internationales Verfahren DCF'].loc[48, 8])              #Instandhaltungsrücklage
    infos.append(excelFile['Internationales Verfahren DCF'].loc[50, 8])              #Capex
    
    return infos

#Objects with wrong valuations
exceptions = ['Commercial_Valuation_Schönebecker Straße 38, 28759 Bremen.xlsm', 
              'Commercial_Valuation_2019_v5.5 - Bornaische Str. 130, 04279 Leipzig - neu.xlsm',
              'Commercial_Valuation_2019_v5_Otto-Hahn-Straße 39, 47167 Duisburg.xlsm',
              'Commercial_Valuation_Template Osterfeldstraße 26 45886 Gelsenkirchen.xlsm',
              'Commercial_Valuation_2019_Karlsruher Straße 82, 69126 Heidelberg.xlsm',
              'Commercial Valuation Heinrich-Zille-Straße 6, 74078 Heilbronn.xlsm',
              'Commercial_Valuation_2019_Johanniterstraße 17-19, 67547 Worms.xlsm',
              'Commercial Valuation Melbeckstraße 18, 42655 Solingen.xlsm',
              'Commercial Valuation Melbeckstraße 18, 42655 Solingen (1).xlsm',
              'Commercial Valuation Katternberger Str. 148, 42655 Solingen.xlsm',
              'Commercial Valuation  Katternberger Straße 150, 42655 Solingen.xlsm',
              'Commercial_Valuation_Template Schützenstraße 102, 42655 Solingen.xlsm',
              'Commercial Valuation Adlerstraße 13, 42655 Solingen.xlsm',
              'Commercial Valuation Neuenweiherstraße, 91056 Erlangen.xlsm']

def roots(city, exceptions):
    counter = 0
    files   = []
    rootDir = 'G:\\Meine Ablage\\Valuation - Commercial\\' + city + '\\'
    print(city)
    for dirName, subdirList, fileList in os.walk(rootDir):
        if (('Datenraum' not in dirName) and ('Briefing' not in dirName) and 
            ('Projektierung' not in dirName) and ('2Pager' not in dirName) and 
            ('Residualwertverfahren' not in dirName)):
            for fname in fileList:
                if ((exceptions[0] or exceptions[1] or exceptions[2] or exceptions[3] or
                     exceptions[4] or exceptions[5] or exceptions[6]  or exceptions[7] or
                     exceptions[8] or exceptions[9] or exceptions[10] or exceptions[11] or
                     exceptions[12] or exceptions[13]) in fname):
                    continue
                if (('Commercial' in fname) and ('Valuation' in fname) and ('.xlsm' in fname) and 
                    (' ' in fname) and ('Projektierung' not in fname) and ('Briefing' not in fname) and 
                    ('.gsheet' not in fname) and ('~' not in fname)):
                    files.append(dirName + '\\' + fname)
                    counter += 1
                    if counter % 50 == 0:
                        print(counter)
    return files 

#List of all cities, where we did valuations (not only in top-100)
def listOfcities(rootDir_):
    subdirList = os.listdir(rootDir_)
    copyDirs   = subdirList.copy()
    newList = [x for x in copyDirs if (("." not in x) and ("New Folder" not in x)
                                      and ("0" not in x) and ("_" not in x))]
    return newList

#Preprocess the dataset with the biggest german cities
def germanCities(data):
    newGerCities = data.copy()
    
    #Some cities with missing information about the number of inhabitants
    specialCities1 = ['Offenbach', 'Weiden', 'Nienburg', 'Verden', 'Köthen', 'Rotenburg', 
                      'Karlstadt', 'Neustadt', 'Homberg', 'Garmisch-Partenkirchen']
    specCitPop1 = [131295, 43052, 31570, 27782, 24876, 22199, 14995, 10177, 13970, 27482]
    
    #Some cities where english city names need to be replaced by german names
    #or cities with incomplete names 
    specialCities2ENG = ['Frankfurt', 'Cologne', 'Munich', 'Nuremberg', 'Ludwigshafen', 'Mülheim', 'Offenbach', 'Esslingen']
    specialCities2GER = ['Frankfurt am Main', 'Köln', 'München', 'Nürnberg', 'Ludwigshafen am Rhein', 
                         'Mülheim an der Ruhr', 'Offenbach am Main', 'Esslingen am Neckar']
    
    #Some provinces where english names need to be replaced by german names
    specialAdmiName1 = ['Bavaria', 'North Rhine-Westphalia', 'Saxony', 'Lower Saxony', 'Saxony-Anhalt', 
                      'Mecklenburg-Western Pomerania', 'Thuringia', 'Rhineland-Palatinate', 'Hesse']
    
    specialAdmiName2 = ['Bayern', 'NRW', 'Sachsen', 'Niedersachsen', 'Sachsen_Anhalt', 'Mecklenburg_Vorpommern',
                        'Thüringen', 'Rheinland_Pfalz', 'Hessen']
    
    #Use these loops to make the corrections listed above
    for i in range(len(specialCities1)):
        newGerCities.loc[newGerCities.city == specialCities1[i], ['population', 'population_proper']] = specCitPop1[i]
        
    for i in range(len(specialCities2ENG)):
        newGerCities.loc[newGerCities.city == specialCities2ENG[i], 'city'] = specialCities2GER[i]    
    
    for i in range(len(specialAdmiName1)):
        newGerCities.loc[newGerCities.admin_name == specialAdmiName1[i], 'admin_name'] = specialAdmiName2[i]
    
    #Sort the cleaned dataframe by population 
    sortedGerCit = newGerCities.sort_values(by = 'population_proper', ascending=False, inplace=False)
    
    return sortedGerCit

#Use this function to add new object in the dictionary 
def add_element(dict, key, value):
    if key not in dict:
        dict[key] = []
    dict[key].append(value)  

#Create a dictionary with rooth to the objects
def dictOfroots(gerCitiesSorted, subdirList):
    d = {}
    for city in gerCitiesSorted.city:
        for cities in subdirList:
            if (city in cities):
                val  = roots(cities, exceptions) 
                add_element(d, cities, val)
    return d

#Write dataframe in the csv file
def writeFiles(data, city):
    path = 'C:\\Users\\User\\Desktop\\Projekt ML\\Interdisziplinäre Projektarbeit\\' + 'data' + city + '.csv'
    data.to_csv(path, index = False)   

In [14]:
#all subdirectories in Valuation Commercial
rootDir    = 'G:\\Meine Ablage\\Valuation - Commercial\\'
subdirList = listOfcities(rootDir)

#dataset of german cities
gerCities       = pd.read_csv('Deutschland_Cities.csv', sep=',')
gerCitiesSorted = germanCities(gerCities)

In [16]:
top1_10      = gerCitiesSorted[2:3]
dictionary1  = dictOfroots(top1_10, subdirList)

Grafing bei München
Garching bei München
Kirchheim bei München
München
50
100
Münchenbernsdorf
Münchener Projekte


In [ ]:
dataset1     = data_reader(dictionary1)

In [26]:
top11_20     = gerCitiesSorted[10:20]
dictionary2  = dictOfroots(top11_20, subdirList)

Bremen
Bremen (1)
50
100
Hannover
Hannover (1)
50
100
Altendorf bei Nürnberg
Altdorf bei Nürnberg
Nürnberg
50
100
Leipzig
50
100
Leipzig (1)
Duisburg
50
100
150
200
250
Bochum
Bochum (1)
50
100
Wuppertal
50
100
150
Bielefeld
Bielefeld (1)
50
Bonn
50
Münster-Sarmsheim
Münstermaifeld
Münster (Hessen)
Münster
50


In [27]:
dataset2     = data_reader(dictionary2)

G:\Meine Ablage\Valuation - Commercial\Bremen\28201\Yorckstr. 61\02 Verkaufsstrategie\Commercial Valuation Yorckstr. 61, 28201 Bremen .xlsm
G:\Meine Ablage\Valuation - Commercial\Bremen (1)\28719 (1)\Stockholmerstrasse 8, 28719 Bremen\Commercial_Valuation_Stockholmerstraße 8, 28719 Bremen_Neue Version.xlsm
G:\Meine Ablage\Valuation - Commercial\Bremen (1)\28719 (1)\Stockholmerstrasse 8, 28719 Bremen\Alt\Commercial_Valuation Stockholmer Straße 8, 28719 Bremen.xlsm
G:\Meine Ablage\Valuation - Commercial\Bremen (1)\28719 (1)\Marßel 86\Einwertung\Commercial_Valuation_2019_v5.5_DE_Alt-Marßel 86, 28719 Bremen.xlsm
G:\Meine Ablage\Valuation - Commercial\Bremen (1)\28719 (1)\Stader Landstraße 16\Einwertung\Commercial_Valuation_2020_v5.5_DE_Stader Landstraße 16, 28719 Bremen Burglesum.xlsm
G:\Meine Ablage\Valuation - Commercial\Bremen (1)\28719 (1)\Burger Heerstraße 37\Commercial_Valuation_Template Burger Heerstraße 37, 28719 Bremen.xlsm
G:\Meine Ablage\Valuation - Commercial\Bremen (1)\28719 (

G:\Meine Ablage\Valuation - Commercial\Bremen (1)\28203 (1)\Kreuzstraße 203\Bewertung\Einwertung Kreuzstraße 41\Commercial_Valuation_2019_4 Kreuzstraße 41.xlsm
G:\Meine Ablage\Valuation - Commercial\Bremen (1)\28203 (1)\Bismarkstraße 20\Bewertung\Einwertung Bismarckstraße 20\Commercial_Valuation_2019_4 Bismarckstraße 20.xlsm
G:\Meine Ablage\Valuation - Commercial\Bremen (1)\28203 (1)\Schönhausenstraße 20\Commercial_Valuation_2019_Schönhausenstraße 20, 28203 Bremen.xlsm
G:\Meine Ablage\Valuation - Commercial\Bremen (1)\28203 (1)\Hemelinger Straße 7\Einwertung Hemelinger Str. 7\Commercial_Valuation_2019_4 Hemelinger Straße 7.xlsm
G:\Meine Ablage\Valuation - Commercial\Bremen (1)\28195 (1)\Rembertiring 21\Einwertung Rembertiring 21\Szenario 1\Commercial_Valuation_2019_4 Rembertiring 21.xlsm
G:\Meine Ablage\Valuation - Commercial\Bremen (1)\28195 (1)\Rembertiring 21\Einwertung Rembertiring 21\Szenario 2\Commercial_Valuation_2019_4 Rembertiring 21, 28195 Bremen.xlsm
G:\Meine Ablage\Valuatio

G:\Meine Ablage\Valuation - Commercial\Hannover (1)\30629 (1)\Gustav-Bratke-Straße 30\Commercial_Valuation_Gustav-Bratke-Straße 30, 30629 Hannover.xlsm
G:\Meine Ablage\Valuation - Commercial\Hannover (1)\30171 (1)\Große Düwelstraße28\Commercial_Valuation Große Düwelstraße 28.xlsm
G:\Meine Ablage\Valuation - Commercial\Hannover (1)\30171 (1)\Große Düwelstraße28\alt\Commercial_Valuation Große Düwelstraße28, 30171 Hannover.xlsm
G:\Meine Ablage\Valuation - Commercial\Hannover (1)\30171 (1)\Große Düwelstraße28\alt\Commercial_Valuation_Große Düwelstraße28, 30171 Hannover.xlsm
G:\Meine Ablage\Valuation - Commercial\Hannover (1)\30171 (1)\Große Düwelstraße28\alt\Commercial_Valuation Große Düwelstraße 28.xlsm
G:\Meine Ablage\Valuation - Commercial\Hannover (1)\30171 (1)\Sallstraße 73\02 Verkaufsstrategie\Commercial Valuation Sallstraße 73, 30171 Hannover.xlsm
G:\Meine Ablage\Valuation - Commercial\Hannover (1)\30451\Fössestraße 61, 30451 Hannover\Commercial_Valuation_Fössestraße 61, 30451 Hanno

G:\Meine Ablage\Valuation - Commercial\Hannover (1)\30655\Roderbruchstraße 10\Einwertung\ALT_Commercial_Valuation_2019_Roderbruchstraße 10, 30655 Hannover.xlsm
G:\Meine Ablage\Valuation - Commercial\Hannover (1)\30655\Roderbruchstraße 10\02 Verkaufsstrategie\Commercial Valuation Roderbruchstraße 10, 30655 Hannover.xlsm
G:\Meine Ablage\Valuation - Commercial\Hannover (1)\30655\Bussestr. 2\Einwertung\Commercial_Valuation_2019_v5.5_DE_Bussestr. 2 30655 Hannover.xlsm
G:\Meine Ablage\Valuation - Commercial\Hannover (1)\30559\Hartmannstraße 4 und 6\Commercial_Valuation_2019_Hartmannstraße 4 und 6, 30559 Hannover.xlsm
G:\Meine Ablage\Valuation - Commercial\Hannover (1)\30559\Gollstraße 67-69\Commercial_Valuation_2019_v5.5_DE_Gollstraße 67-69, 30559 Hannover.xlsm
G:\Meine Ablage\Valuation - Commercial\Hannover (1)\30559\Ernststraße 21\02 Verkaufsstrategie\Commercial Valuation Ernststraße 21, 30559 Hannover.xlsm
G:\Meine Ablage\Valuation - Commercial\Hannover (1)\30559\Wasserkampstraße 8\02 Ver

G:\Meine Ablage\Valuation - Commercial\Hannover (1)\30161\Lenbachstr. 61\02 Verkaufsstrategie\Commercial Valuation  Lenbachstr. 61, 30161  Hannover.xlsm
G:\Meine Ablage\Valuation - Commercial\Hannover (1)\30853\Buschkamp 13\02 Verkaufsstrategie\Commercial_Valuation_Template Buschkamp 13, 30853 Hannover.xlsm
G:\Meine Ablage\Valuation - Commercial\Altendorf bei Nürnberg\90518\Rathausgasse 4\02 Verkaufsstrategie\Commercial Valuation Rathausgasse 4, 90518 Altendorf bei Nürnberg.xlsm
G:\Meine Ablage\Valuation - Commercial\Altdorf bei Nürnberg\90518 (1)\Emil-von-Stromer-Platz 4\Commercial_Valuation_2019_v5.5_Emil-von-Stromer-Platz 4, 90518 Altdorf bei Nürnberg.xlsm
G:\Meine Ablage\Valuation - Commercial\Altdorf bei Nürnberg\90518 (1)\Emil-von-Stromer-Platz 4\Verkaufsstrategie\Commercial_Valuation_2019_v5.5_Emil-von-Stromer-Platz 4, 90518 Altdorf bei Nürnberg.xlsm
G:\Meine Ablage\Valuation - Commercial\Altdorf bei Nürnberg\90518 (1)\Rathausgasse 4\02 Verkaufsstrategie\Commercial Valuation  Ra

...

G:\Meine Ablage\Valuation - Commercial\Wuppertal\42277 (1)\Schwarzbach 62\Verkaufsstrategie\Commercial_Valuation_2019_v5.5_Schwarzbach 62, 42277 Wuppertal.xlsm
G:\Meine Ablage\Valuation - Commercial\Wuppertal\42277 (1)\Berliner Str. 137\Einwertung\Commercial_Valuation_2019_v5.5_DE_Berliner Str. 137, 42277 Wuppertal.xlsm
G:\Meine Ablage\Valuation - Commercial\Wuppertal\42277 (1)\Germanenstraße 57\Bewertung\Commercial_Valuation_2019_v5.5_DE Germanenstraße 57, 42277 Wuppertal.xlsm
G:\Meine Ablage\Valuation - Commercial\Wuppertal\42277 (1)\Huldastraße 41A\Einwertung\Commercial_Valuation_2019_v5.5_DE_Übergangsversion_Huldastraße 41A, 42277 Wuppertal.xlsm
G:\Meine Ablage\Valuation - Commercial\Wuppertal\42277 (1)\Bartholomäusstr. 56\02 Verkaufsstrategie\Commercial Valuation Bartholomäusstr. 56, 42277  Wuppertal.xlsm
G:\Meine Ablage\Valuation - Commercial\Wuppertal\42289\Rauental 49\Commercial_Valuation_2020_v5.5_DE_Rauental 49, 42289 Wuppertal.xlsm
G:\Meine Ablage\Valuation - Commercial\Wupp

G:\Meine Ablage\Valuation - Commercial\Bielefeld (1)\33689\Hebronweg 3\Bewertung alt 01 2020\Commercial_Valuation_2019_Hebronweg 3, 33689 Bielefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Bielefeld (1)\33689\Hebronweg 3\Bewertung alt 01 2020\Einwertung\Commercial_Valuation_2019_Hebronweg 3, 33689 Bielefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Bielefeld (1)\33689\Hebronweg 3\Einwertung neu\Commercial_Valuation_2019_v5.5_DE_Übergangsversion_Hebronweg 3, 33689 Bielefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Bielefeld (1)\33689\Alte Verlerstraße 1\02 Verkaufsstrategie\Commercial Valuation Alte Verlerstraße 1, 33689 Bielefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Bielefeld (1)\33607\Heeper Straße 62\Commercial_Valuation_2019_Heeper Str. 62, 33607 Bielefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Bielefeld (1)\33607\Heeper Straße 62\Alt\Commercial_Valuation_2019_Heeper Str. 62, 33607 Bielefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Bielefeld (1)\33607\Bleichst

G:\Meine Ablage\Valuation - Commercial\Bielefeld (1)\33609\Schuckenbaumer Straße 16\02 Verkaufsstrategie\Commercial Valuation Schuckenbaumer Straße 16, 33609 Bielefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Bielefeld (1)\33609\Schuckenbaumer Straße 16\02 Verkaufsstrategie\Commercial_Valuation_Template (version 1).xlsm
G:\Meine Ablage\Valuation - Commercial\Bielefeld (1)\33609\Rabenhof 69A\02 Verkaufsstrategie\Commercial_Valuation_Template Rabenhof 69A, 33609 Bielefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Bielefeld (1)\33619\Am Rehhagen 4\02 Verkaufsstrategie\Commercial Valuation Frohnauer Straße 8, 33619 Bielefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Bonn\53121\Röckumstraße 24\Commercial_Valuation_Röckumstr. 24, 53121 Bonn.xlsm
G:\Meine Ablage\Valuation - Commercial\Bonn\53111 (1)\Wilhelmstraße 54\Commercial_Valuation - Wilhelmstraße 54, 53111 Bonn.xlsm
G:\Meine Ablage\Valuation - Commercial\Bonn\53111 (1)\Friedrichstraße 7\Commercial_Valuation_ Friedrichstraße 7, 53

G:\Meine Ablage\Valuation - Commercial\Bonn\53229\Marktstraße 97\02 Verkaufsstrategie\Commercial Valuation Marktstraße 97, 53229 Bonn.xlsm
G:\Meine Ablage\Valuation - Commercial\Bonn\53229\Marktstraße 97\02 Verkaufsstrategie\Neu\Commercial Valuation Marktstraße 97, 53229 Bonn.xlsm
G:\Meine Ablage\Valuation - Commercial\Bonn\53229\Marktstraße 97\02 Verkaufsstrategie\Neu\Lange Version\Commercial Valuation Marktstraße 97, 53229 Bonn.xlsm
G:\Meine Ablage\Valuation - Commercial\Bonn\53115\Dechenstraße 7\02 Verkaufsstrategie\Commercial Valuation Dechenstraße 7, 53115 Bonn neu.xlsm
G:\Meine Ablage\Valuation - Commercial\Bonn\53115\Dechenstraße 7\02 Verkaufsstrategie\Alt\Commercial Valuation Dechenstraße 7, 53115 Bonn.xlsm
G:\Meine Ablage\Valuation - Commercial\Bonn\53115\Mozartstraße 44\02 Verkaufsstrategie\Commercial_Valuation_Template_Mozartstraße 44, 53115 Bonn.xlsm
G:\Meine Ablage\Valuation - Commercial\Bonn\53115\Am Botanischen Garten 2\02 Verkaufsstrategie\Commercial_Valuation_Template 

G:\Meine Ablage\Valuation - Commercial\Münster\48147 (1)\Kettelerstraße 8\02 Verkaufsstrategie\Commercial Valuation Kettelerstraße 8, 48147 Münster (1).xlsm
G:\Meine Ablage\Valuation - Commercial\Münster\48147 (1)\Rudolf-von-Langen-Straße 53\02 Verkaufsstrategie\Commercial_Valuation_Template Rudolf-von-Langen-Straße 53, 48147 Münster.xlsm
G:\Meine Ablage\Valuation - Commercial\Münster\48147 (1)\Rumphorstweg 39\02 Verkaufsstrategie\Commercial_Valuation_Template Rumphorstweg 39, 48147 Münster.xlsm
G:\Meine Ablage\Valuation - Commercial\Münster\48167 (1)\Münsterstr. 8-10\Commercial_Valuation_2019_v5.5_DE_Münsterstr. 8-10, 48167 Münster.xlsm
G:\Meine Ablage\Valuation - Commercial\Münster\48167 (1)\Zum Erlenbusch 1a\02 Verkaufsstrategie\Commercial_Valuation_Template Zum Erlenbusch 1a, 48167 Münster.xlsm
G:\Meine Ablage\Valuation - Commercial\Münster\48151\Scharnhorststraße 59\02 Verkaufsstrategie\Commercial Valuation Scharnhorststraße 59, 48151 Münster.xlsm
G:\Meine Ablage\Valuation - Comme

In [28]:
top21_30     = gerCitiesSorted[20:30]
dictionary3  = dictOfroots(top21_30, subdirList)

Karlsruhe
50
Mannheim- Feudenheim
Mannheim
50
100
Augsburg
Augsburg (1)
Wiesbaden
50
Mönchengladbach
50
Mönchengladbach (1)
50
Gelsenkirchen
Gelsenkirchen (1)
50
100
Braunschweig
Kiel
50
Aachen
Aachen (1)
50
Chemnitz
50


In [29]:
dataset3     = data_reader(dictionary3)

G:\Meine Ablage\Valuation - Commercial\Karlsruhe\76149\Mitteltorstraße 51\Commercial_Valuation Mitteltorstraße 51, 76149 Karlsruhe OT Neureut.xlsm
G:\Meine Ablage\Valuation - Commercial\Karlsruhe\76149\Mitteltorstraße 51\alt\Commercial_Valuation Mitteltorstraße 51, 76149 Karlsruhe OT Neureut.xlsm
G:\Meine Ablage\Valuation - Commercial\Karlsruhe\76149\Am Sandfeld 11A\Commercial_Valuation_Am Sandfeld 11A,76149 Karlsruhe.xlsm
G:\Meine Ablage\Valuation - Commercial\Karlsruhe\76149\Unterer Dammweg 7a\02 Verkaufsstrategie\Unterer Dammweg 7a, 76149 Karlsruhe - Marcel\Option 1\Commercial_Valuation_Unterer Dammweg 7a, 76149 Karlsruhe_Option 1.xlsm
G:\Meine Ablage\Valuation - Commercial\Karlsruhe\76149\Unterer Dammweg 7a\02 Verkaufsstrategie\Unterer Dammweg 7a, 76149 Karlsruhe - Marcel\Option 1\Short\Commercial_Valuation_Unterer Dammweg 7a, 76149 Karlsruhe_Option 1.xlsm
G:\Meine Ablage\Valuation - Commercial\Karlsruhe\76149\Unterer Dammweg 7a\02 Verkaufsstrategie\Unterer Dammweg 7a, 76149 Karlsr

G:\Meine Ablage\Valuation - Commercial\Karlsruhe\76185 (1)\Grillparzerstraße 2\02 Verkaufsstrategie\Commercial_Valuation_Template Grillparzerstraße 2, 76185 Karlsruhe.xlsm
G:\Meine Ablage\Valuation - Commercial\Karlsruhe\76185 (1)\Binger Straße 87\02 Verkaufsstrategie\Commercial Valuation Binger Straße 87, 76185 Karlsruhe.xlsm
G:\Meine Ablage\Valuation - Commercial\Karlsruhe\76137\Augartenstraße 85A\Commercial_Valuation_2019_v5.5_Augartenstraße 85, 76137 Karlsruhe.xlsm
G:\Meine Ablage\Valuation - Commercial\Karlsruhe\76137\Augartenstraße 85A\alt\Commercial_Valuation_2019_v5.5_Augartenstraße 85, 76137 Karlsruhe.xlsm
G:\Meine Ablage\Valuation - Commercial\Karlsruhe\76137\Winterstraße 48\02 Verkaufsstrategie\Commercial Valuation Winterstraße 48, 76137 Karlsruhe.xlsm
G:\Meine Ablage\Valuation - Commercial\Karlsruhe\76137\Wielandtstraße 26\02 Verkaufsstrategie\Commercial Valuation Wielandtstraße 26, 76137 Karlsruhe.xlsm
G:\Meine Ablage\Valuation - Commercial\Karlsruhe\76137\Wielandtstraße 2

G:\Meine Ablage\Valuation - Commercial\Mannheim\68169\Mittelstraße 28A\Commercial_Valuation_2019_v5.5_DE Mittelstraße 28A, 68169 Mannheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Mannheim\68169\Pumpwerkstraße 23\Einwertung\Commercial_Valuation_2020_v5.5_Pumpwerkstraße 23, 68169 Mannheim.xlsm.xlsm
G:\Meine Ablage\Valuation - Commercial\Mannheim\68169\Friesenheimer Str. 26\Commercial_Valuation_2019_v5.5_DE_Übergangsversion_Friesenheimer Str. 26, 68169 Mannheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Mannheim\68169\Elfenstraße 12\Commercial_Valuation_Template_Elfenstraße 12, 68169 Mannheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Mannheim\68165\Augustaanlage 15\Verkaufsstrategie\Commercial_Valuation_Augustaanlage 15, 68165 Mannheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Mannheim\68165\Keplerstreaße 39\Commercial_Valuation_2019_3_Keplerstraße 39, 68165 Mannheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Mannheim\68165\Keplerstreaße 39\alt\Commercial_Valuation_2019_3_Keplers

G:\Meine Ablage\Valuation - Commercial\Mannheim\68161 (1)\M2 11\Einwertung\Commercial_Valuation_2019_v5.5_DE_M2 11, 68161 Mannheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Mannheim\68161 (1)\M2 11\Einwertung\Alte Version\Commercial_Valuation_2019_v5.5_DE_M2 11, 68161 Mannheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Mannheim\68161 (1)\Q3\02 Verkaufsstrategie\Commercial Valuation Q3, 68161 Mannheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Mannheim\68161 (1)\L7\02 Verkaufsstrategie\Commercial Valuation L7, 68161 Mannheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Mannheim\68259\Hölderlinstraße 20\Verkaufsstrategie\Commercial_Valuation_Hölderlinstraße 20, 68259 Mannheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Mannheim\68259\Schwanenstraße 40\02 Verkaufsstrategie\Commercial_Valuation_Schwanenstraße 40, 68259 Mannheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Mannheim\68259\Pfalzstraße 1\02 Verkaufsstrategie\Commercial Valuation Pfalzstraße 1, 68259 Mannheim.xlsm
G:\Meine Ab

G:\Meine Ablage\Valuation - Commercial\Wiesbaden\55252\Sankt-Florian-Straße 6\alt\Commercial_Valuation_Sankt Florian Str. 6, Wiesbaden.xlsm
G:\Meine Ablage\Valuation - Commercial\Wiesbaden\55252\In der Witz 29\Bewertung\Commercial_Valuation_2019_v5.5_DE_In der Witz 29, 55252 Wiesbaden.xlsm
G:\Meine Ablage\Valuation - Commercial\Wiesbaden\55252\Josefinenanlage 9\Josefinenanlage 9\Commercial_Valuation_Template Josefinenanlage 9, 55252 Wiesbaden.xlsm
G:\Meine Ablage\Valuation - Commercial\Wiesbaden\55252\Josefinenanlage 9\Blücherstraße 7\Commercial_Valuation_Template Blücherstraße 7.xlsm
G:\Meine Ablage\Valuation - Commercial\Wiesbaden\55252\Boelckestraße 35\02 Verkaufsstrategie\Commercial_Valuation_Template Boelckestraße 35, 55252 Wiesbaden.xlsm
G:\Meine Ablage\Valuation - Commercial\Wiesbaden\55252\Berstädter Grabenweg 12\02 Verkaufsstrategie\Commercial Valuation Berstädter Grabenweg 12, 55252 Wiesbaden (1).xlsm
G:\Meine Ablage\Valuation - Commercial\Wiesbaden\55252\Berstädter Grabenweg

...

G:\Meine Ablage\Valuation - Commercial\Braunschweig\38106\Wodanstraße 35\02 Verkaufsstrategie\Commercial_Valuation_Template Wodanstraße 35, 38106 Braunschweig.xlsm
G:\Meine Ablage\Valuation - Commercial\Braunschweig\38118\Sophienstraße 12\02 Verkaufsstrategie\Commercial_Valuation_Sophienstraße 12, 38118 Braunschweig.xlsm
G:\Meine Ablage\Valuation - Commercial\Braunschweig\38118\Helenenstraße 15-16\02 Verkaufsstrategie\Commercial_Valuation_Template_Helenenstraße 15-16, 38118 Braunschweig.xlsm
G:\Meine Ablage\Valuation - Commercial\Braunschweig\38118\Helenenstraße 15-16\02 Verkaufsstrategie\Commercial_Valuation_Template_Helenenstraße 15-16, 38118 Braunschweig projektierung.xlsm
G:\Meine Ablage\Valuation - Commercial\Braunschweig\38118\Höfenstraße 16\02 Verkaufsstrategie\Commercial Valuation Höfenstraße 16, 38118 Braunschweig.xlsm
G:\Meine Ablage\Valuation - Commercial\Braunschweig\38118\Kreuzstraße 41\02 Verkaufsstrategie\Commercial_Valuation_Template Kreuzstraße 41, 38118 Braunschweig.x

G:\Meine Ablage\Valuation - Commercial\Kiel\24113\Rendsburger Landstraße 86 u. 86b\Einwertung\Commercial_Valuation_2019_v5.5_DE_Rendsburger Landstraße 86 u. 86b, 24113 Kiel.xlsm
G:\Meine Ablage\Valuation - Commercial\Kiel\24113\Rendsburger Landstraße 86 u. 86b\Einwertung\alt\Commercial_Valuation_2019_v5.5_DE_Rendsburger Landstraße 86 u. 86b, 24113 Kiel.xlsm
G:\Meine Ablage\Valuation - Commercial\Kiel\24113\Rendsburger Landstraße 85\Einwertung\Commercial_Valuation_Rendsburger Landstraße 85, 24113 Kiel.xlsm
G:\Meine Ablage\Valuation - Commercial\Kiel\24113\Danewerkstraße 9A\02 Verkaufsstrategie\Commercial_Valuation_Template Danewerkstraße 9A, 24113 Kiel.xlsm
G:\Meine Ablage\Valuation - Commercial\Kiel\24146\Preetzer Chaussee 144\Einwertung\Commercial_Valuation_2020_v5.5_Preetzer Chaussee 144, 24146 Kiel.xlsm
G:\Meine Ablage\Valuation - Commercial\Kiel\24146\Dorfstraße 15\02 Verkaufsstrategie\Commercial_Valuation_Template Dorfstraße 15, 24146 Kiel.xlsm
G:\Meine Ablage\Valuation - Commerci

G:\Meine Ablage\Valuation - Commercial\Aachen (1)\52070\Jülicher Straße 45A\02 Verkaufsstrategie\Commercial_Valuation_Template Jülicher Straße 45A, 52070 Aachen.xlsm
G:\Meine Ablage\Valuation - Commercial\Aachen (1)\52068\Auf der Hüls 190\02 Verkaufsstrategie\Commercial_Valuation_Auf der Hüls 190, 52068 Aachen.xlsm
G:\Meine Ablage\Valuation - Commercial\Aachen (1)\52068\Zeppelinstraße 62\02 Verkaufsstrategie\Commercial Valuation Zeppelinstraße 62, 52068 Aachen.xlsm
G:\Meine Ablage\Valuation - Commercial\Aachen (1)\52068\Zeppelinstraße 62\02 Verkaufsstrategie\Commercial_Valuation Zeppelinstraße 62, 52068 Aachen.xlsm
G:\Meine Ablage\Valuation - Commercial\Aachen (1)\52064\Ludwigsallee 9\02 Verkaufsstrategie\Commercial_Valuation_Ludwigsallee 9, 52062 Aachen.xlsm
G:\Meine Ablage\Valuation - Commercial\Aachen (1)\52064\Turmstraße 188\02 Verkaufsstrategie\Commercial_Valuation_Turmstraße 188, 52064 Aachen.xlsm
G:\Meine Ablage\Valuation - Commercial\Aachen (1)\52064\Paulusstraße\02 Verkaufsstr

G:\Meine Ablage\Valuation - Commercial\Chemnitz\09116\Stelzendorfer Straße 133\02 Verkaufsstrategie\Commercial_Valuation_Template Stelzendorfer Str. 133, 09116 Chemnitz.xlsm
G:\Meine Ablage\Valuation - Commercial\Chemnitz\09116\Stelzendorfer Straße 133\02 Verkaufsstrategie\Alt\Commercial_Valuation_Template Stelzendorfer Str. 133, 09116 Chemnitz.xlsm
G:\Meine Ablage\Valuation - Commercial\Chemnitz\09116\Matthesstraße 130\02 Verkaufsstrategie\Commercial Valuation Matthesstraße 130, 09116 Chemnitz.xlsm
G:\Meine Ablage\Valuation - Commercial\Chemnitz\09116\Matthesstraße 130\02 Verkaufsstrategie\Commercial_Valuation Matthesstraße 130, 09113 Chemnitz.xlsm
G:\Meine Ablage\Valuation - Commercial\Chemnitz\09116\Einsteinstr 6\02 Verkaufsstrategie\Commercial Valuation Einsteinstr 6, 09116 Chemnitz.xlsm
G:\Meine Ablage\Valuation - Commercial\Chemnitz\09116\Lützowstraße 13\02 Verkaufsstrategie\Commercial Valuation Lützowstraße 13, 09116 Chemnitz.xlsm
G:\Meine Ablage\Valuation - Commercial\Chemnitz\

G:\Meine Ablage\Valuation - Commercial\Chemnitz\09111\Augustusburger Straße 43\02 Verkaufsstrategie\Commercial Valuation Augustusburger Straße 43, 09111 Chemnitz.xlsm
G:\Meine Ablage\Valuation - Commercial\Chemnitz\09123 (1)\Klaffenbacher Hauptstraße 110\02 Verkaufsstrategie\Commercial Valuation Klaffenbacher Hauptstraße 110, 09123 Chemnitz.xlsm


In [30]:
top31_40     = gerCitiesSorted[30:40]
dictionary4  = dictOfroots(top31_40, subdirList)

Halle (Westfalen)
Hallerndorf
Steinbach-Hallenberg
Hallein
Halle
Hallenberg
Halle (Saale)
Magdeburg
Burg (bei Magdeburg)
Magdeburg (2)
50
Freiburg im Breisgau
Kölner Krefeld
Krefeld
50
Krefeld (1)
Lübeck
Oberhausen
50
100
Rostock
Rostock (1)
Kassel
50
Hagenow
Hagenbüchach
Hagenburg
Hagenbach
Hagen
50
Mainz


In [31]:
dataset4     = data_reader(dictionary4)

G:\Meine Ablage\Valuation - Commercial\Halle (Westfalen)\33790\Lange Straße 9\02 Verkaufsstrategie\Commercial Valuation Lange Straße 9, 33790 Halle (Westfalen).xlsm
G:\Meine Ablage\Valuation - Commercial\Halle (Westfalen)\33790\Berliner Straße 19\02 Verkaufsstrategie\Commercial_Valuation_Berliner Straße 19, 33790 Halle (Westfalen).xlsm
G:\Meine Ablage\Valuation - Commercial\Halle (Westfalen)\33790\Weidenstraße 5\02 Verkaufsstrategie\Commercial Valuation Weidenstraße 5, 33790 Halle (Westfalen).xlsm
G:\Meine Ablage\Valuation - Commercial\Halle (Westfalen)\33790\Weidenstraße 5\02 Verkaufsstrategie\Commercial_Valuation_Template_Weidenstraße 5, 33790 Halle (Westfalen).xlsm
G:\Meine Ablage\Valuation - Commercial\Halle\06114\Fischer-von-Erlach-Straße 77\02 Verkaufsstrategie\Commercial_Valuation_Fischer-von-Erlach-Straße 77, 06114 Halle (Saale).xlsm
G:\Meine Ablage\Valuation - Commercial\Halle\06110\Turmstrasse 59D\02 Verkaufsstrategie\Commercial Valuation Turmstrasse 59D, 06110 Halle.xlsm
G:\

G:\Meine Ablage\Valuation - Commercial\Burg (bei Magdeburg)\39288\Grünstraße 13A\02 Verkaufsstrategie\neu\Commercial_Valuation_Template_Grünstraße 13A, 39288 Burg (bei Magdeburg).xlsm
G:\Meine Ablage\Valuation - Commercial\Burg (bei Magdeburg)\39288\Blumenthaler Straße 41\Bewertung\Commercial_Valuation_2019_v5.5_DE_Blumenthaler Straße 41, 39288 Burg (bei Magdeburg).xlsm
G:\Meine Ablage\Valuation - Commercial\Burg (bei Magdeburg)\39288\Schartauer Straße 38\Einwertung\Commercial_Valuation Schartauer Straße 38, 39288 Burg (bei Magdeburg).xlsm
G:\Meine Ablage\Valuation - Commercial\Burg (bei Magdeburg)\39288\Deichstraße 23\02 Verkaufsstrategie\Commercial Valuation Deichstraße 23, 39288 Burg (bei Magdeburg).xlsm
G:\Meine Ablage\Valuation - Commercial\Magdeburg (2)\39110\Ummendorfer Straße 9\Commercial_Valuation_Ummendorfer Straße 9_Magdeburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Magdeburg (2)\39110\Ummendorfer Straße 9\Alt\Commercial_Valuation_Ummendorfer Straße 9_Magdeburg.xlsm
G:\Me

G:\Meine Ablage\Valuation - Commercial\Magdeburg (2)\39112 (1)\Braunschweiger Straße 15\02 Verkaufsstrategie\Final\Commercial_Valuation_Braunschweiger Straße 15, 39112 Magdeburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Magdeburg (2)\39112 (1)\Lemsdorfer Weg 3\02 Verkaufsstrategie\Commercial Valuation Lemsdorfer Weg 3, 39112 Magdeburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Magdeburg (2)\39112 (1)\Halberstädter Str. 192\02 Verkaufsstrategie\Commercial Valuation Halberstädter Str. 192, 39112 Magdeburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Magdeburg (2)\39112\Ackerstraße 18\Commercial_Valuation_2019_v5.5_Ackerstraße 18, 39112 Magdeburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Magdeburg (2)\39114\Weberstraße 22\Einwertung\Commercial_Valuation_2019_v5.5_Weberstraße 22, 39114 Magdeburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Magdeburg (2)\39114\Weberstraße 22\Einwertung\alt\Commercial_Valuation_2019_v5.5_Weberstraße 22, 39114 Magdeburg.xlsm
G:\Meine Ablage\Valuation - Com

G:\Meine Ablage\Valuation - Commercial\Krefeld\47798 (2)\Westwall 30\02 Verkaufsstrategie\Commercial_Valuation_Template_ Westwall 30, 47798 Krefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Krefeld\47798 (2)\Sankt-Anton-Straße 216\02 Verkaufsstrategie\Commercial Valuation Sankt-Anton-Straße 216, 47798 Krefeld (1).xlsm
G:\Meine Ablage\Valuation - Commercial\Krefeld\47798 (2)\Sankt-Anton-Straße 216\02 Verkaufsstrategie\Commercial Valuation Sankt-Anton-Straße 216, 47798 Krefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Krefeld\47798 (2)\Neusser Straße 51\02 Verkaufsstrategie\Commercial_Valuation_Template Neusser Straße 51, 47798 Krefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Krefeld\47798 (2)\Ostwall 46\02 Verkaufsstrategie\Commercial Valuation Ostwall 46, 47798 Krefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Krefeld\47798 (2)\Rheinstr. 95\02 Verkaufsstrategie\Commercial Valuation Rheinstr. 95, 47798 Krefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Krefeld\47798 (2)\Gladba

G:\Meine Ablage\Valuation - Commercial\Krefeld\47809\Glockenspitz 329\02 Verkaufsstrategie\Commercial_Valuation_Template_ Glockenspitz 329, 47809 Krefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Krefeld\47809\Düsseldorfer Straße 219\02 Verkaufsstrategie\Commercial Valuation Düsseldorfer Straße 219, 47809 Krefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Krefeld\47799\Dießemer Str. 80\Commercial_Valuation_2019_v5.5_DE Dießemer Str. 80, 47799 Krefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Krefeld\47799\Dießemer Str. 80\Verkaufsstrategie\Commercial_Valuation_Diessemer Straße 48, 47799 Krefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Krefeld\47799\Dießemer Strasse 48\02 Verkaufsstrategie\Commercial Valuation Dießemer Strasse 48, 47799  Krefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Krefeld\47798 (1)\Münkerstr. 55\Commercial_Valuation_Münkerstraße 55, 47798 Krefeld.xlsm
G:\Meine Ablage\Valuation - Commercial\Krefeld\47798 (1)\Garnstraße 61\02 Verkaufsstrategie\Commercial_V

...

G:\Meine Ablage\Valuation - Commercial\Oberhausen\46149 (2)\Lindnerstraße 191\Commercial_Valuation_2019_v5.5_DE Lindnerstr. 191, 46149 Oberhausen.xlsm
G:\Meine Ablage\Valuation - Commercial\Oberhausen\46117\Gildenstraße 24\Commercial_Valuation_2019_Gildenstraße 24, 46117 Oberhausen.xlsm
G:\Meine Ablage\Valuation - Commercial\Oberhausen\46117\Einbleckstraße 55\Commercial_Valuation_2019_v5.5 Einbleckstraße 55, 46117 Oberhausen.xlsm
G:\Meine Ablage\Valuation - Commercial\Oberhausen\46149 (1)\Moosstraße 14\Commercial_Valuation_2019_v5.5_DE Moosstraße 14, 46149 Oberhausen.xlsm
G:\Meine Ablage\Valuation - Commercial\Oberhausen\46149 (1)\Kiebitzstraße 20\Einwertung\Commercial_Valuation_2019_v5.5_Kiebitzstraße 20, 46149 Oberhausen.xlsm
G:\Meine Ablage\Valuation - Commercial\Oberhausen\46149 (1)\Kiebitzstraße 20\Einwertung\alt\Commercial_Valuation_2019_v5.5_Kiebitzstraße 20, 46149 Oberhausen.xlsm
G:\Meine Ablage\Valuation - Commercial\Oberhausen\46149 (1)\Hagenstraße 14\Commercial_Valuation_ÜV_

G:\Meine Ablage\Valuation - Commercial\Rostock (1)\18119\Am Strom 71\Einwertung\Commercial_Valuation_2019_v5.5_Am Strom 71, 18119 Warnemünde.xlsm
G:\Meine Ablage\Valuation - Commercial\Rostock (1)\18147\Zum Fohlenhof 6 12\Commercial_Valuation_2019_v5.5_DE Zum Fohlenhof 6-12, 18147 Rostock.xlsm
G:\Meine Ablage\Valuation - Commercial\Rostock (1)\18057 (1)\Hundertmännerstraße 3\02 Verkaufsstrategie\Commercial Valuation Hundertmännerstraße 3, 18057 Rostock.xlsm
G:\Meine Ablage\Valuation - Commercial\Rostock (1)\18057 (1)\Am Kabutzenhof 37\Einwertung\Commercial_Valuation_2020_v5.5_DE_Am Kabutzenhof 37, 18057 Rostock.xlsm
G:\Meine Ablage\Valuation - Commercial\Rostock (1)\18057 (1)\Ulmenstraße 28\Einwertung\Commercial_Valuation_2019_v5.5_DE_Ulmenstraße 28, 18057 Rostock.xlsm
G:\Meine Ablage\Valuation - Commercial\Rostock (1)\18057 (1)\Dethardingstraße 24\Commercial_Valuation_2019_v5.5_Dethardingstraße 24, 18057 Rostock.xlsm
G:\Meine Ablage\Valuation - Commercial\Rostock (1)\18057 (1)\Rembran

G:\Meine Ablage\Valuation - Commercial\Kassel\34128 (1)\Heisestraße 1 + 3\02 Verkaufsstrategie\Commercial_Valuation_Template Heisestraße 1 + 3, 34128 Kassel.xlsm
G:\Meine Ablage\Valuation - Commercial\Kassel\34134\Frankfurter Straße 269\Commercial_Valuation_2019_v5.5 Frankfurter Str. 269, 34134 Kassel.xlsm
G:\Meine Ablage\Valuation - Commercial\Kassel\34127\Holländische Straße 159-161\Commercial_Valuation_2019_v5.5_DE Holländische Straße 159-161, 34127 Kassel.xlsm
G:\Meine Ablage\Valuation - Commercial\Kassel\34127\Fiedlerstraße 186\Commercial_Valuation_Fiedlerstraße 186, 34127 Kassel.xlsm
G:\Meine Ablage\Valuation - Commercial\Kassel\34117\Bremer Straße 28\02 Verkaufsstrategie\Commercial Valuation Bremer Straße 28, 34117  Kassel.xlsm
G:\Meine Ablage\Valuation - Commercial\Kassel\34130\Zentgrafenstraße 23\Einwertung\Commercial_Valuation_2020_v5.5_DE_Zentgrafenstraße 23 & 23a, 34130 Kassel.xlsm
G:\Meine Ablage\Valuation - Commercial\Kassel\34132\Mattenbergstraße 89\Commercial_Valuation_

G:\Meine Ablage\Valuation - Commercial\Hagen\58089 (1)\Lange Straße 72B\02 Verkaufsstrategie\Commercial Valuation Lange Straße 72B, 58089 Hagen.xlsm
G:\Meine Ablage\Valuation - Commercial\Hagen\58089 (1)\Reichsbahnstraße 5\Commercial_Valuation_2019_Reichsbahnstraße 5, 58089 Hagen.xlsm
G:\Meine Ablage\Valuation - Commercial\Hagen\58089 (1)\Wehringhauser Str. 63\Verkaufsstrategie\Commercial_Valuation_2019_v5_Wehringhauser Str. 63, 58089 Hagen.xlsm
G:\Meine Ablage\Valuation - Commercial\Hagen\58089 (1)\Wehringhauser Str. 63\Verkaufsstrategie\alt\Commercial_Valuation_2019_v5_Wehringhauser Str. 63, 58089 Hagen.xlsm
G:\Meine Ablage\Valuation - Commercial\Hagen\58089 (1)\Lange Straße 48\Commercial_Valuation_Template_Lange Straße 48, 58089 Hagen.xlsm
G:\Meine Ablage\Valuation - Commercial\Hagen\58089 (1)\Tunnelstraße 2\02 Verkaufsstrategie\Commercial Valuation Tunnelstraße 2, 58089 Hagen.xlsm
G:\Meine Ablage\Valuation - Commercial\Hagen\58089 (1)\Wielandstraße 10\02 Verkaufsstrategie\Commercia

G:\Meine Ablage\Valuation - Commercial\Mainz\55120\Wöhlerstraße 8\02 Verkaufsstrategie\Commercial Valuation Wöhlerstraße 8, 55120 Mainz (1).xlsm
G:\Meine Ablage\Valuation - Commercial\Mainz\55120\Hauptstraße 185\02 Verkaufsstrategie\Commercial Valuation Hauptstraße 185, 55120 Mainz.xlsm
G:\Meine Ablage\Valuation - Commercial\Mainz\55128\Domherrngasse 17\02 Verkaufsstrategie\Domherrngasse 17, 55128 Mainz\Commercial Valuation Domherrngasse 17, 55128 Mainz.xlsm
G:\Meine Ablage\Valuation - Commercial\Mainz\55128\Domherrngasse 17\02 Verkaufsstrategie\Waldalgesheimer Str. 14H, 55545 Bad Kreuznach\Commercial Valuation Waldalgesheimer Str. 14H, 55545 Bad Kreuznach.xlsm
G:\Meine Ablage\Valuation - Commercial\Mainz\55128\Domherrngasse 17\02 Verkaufsstrategie\Wilhelmsstraße 26, 55128 Mainz\Commercial Valuation Wilhelmsstraße 26, 55128 Mainz.xlsm
G:\Meine Ablage\Valuation - Commercial\Mainz\55128\Domherrngasse 17\02 Verkaufsstrategie\Lannerstraße 59-61, 55270 Ober-Olm\Commercial Valuation Lannerst

In [32]:
top41_50     = gerCitiesSorted[40:50]
dictionary5  = dictOfroots(top41_50, subdirList)

Saarbrücken-Klarental
Saarbrücken
Saarbrücken (1)
50
Bockenhöfel   Hamm
Hamminkeln
Hammerstein
Hammelburg
Hamm
50
Hammersbach
Potsdam
Erfurt
Ludwigshafen am Rhein
Ludwigshafen
Bodman-Ludwigshafen
Ludwigshafen am Rhein (1)
Mülheim
Mülheim an der Ruhr
50
Oldenburg bei Ranzin
Cappeln (Oldenburg)
Steinfeld (Oldenburg)
Oldenburg
Osnabrück
50
Leverkusen
Heidelberg


In [33]:
dataset5     = data_reader(dictionary5)

G:\Meine Ablage\Valuation - Commercial\Saarbrücken\66130\Saargemünder Str. 106\02 Verkaufsstrategie\Commercial Valuation Saargemünder Str. 106, 66130 Saarbrücken.xlsm
G:\Meine Ablage\Valuation - Commercial\Saarbrücken (1)\66125\Bahnhofstraße 69\Commercial_Valuation_Bahnhofstraße 69_Saarbrücken.xlsm
G:\Meine Ablage\Valuation - Commercial\Saarbrücken (1)\66125\Bahnhofstraße 69\alt\Commercial_Valuation Bahnhofstraße 69, 66125 Saarbrücken.xlsm
G:\Meine Ablage\Valuation - Commercial\Saarbrücken (1)\66125\Saarbrücker Str. 357\Einwertung\Commercial_Valuation_2019_Saarbrücker Str. 357, 66125 Saarbrücken.xlsm
G:\Meine Ablage\Valuation - Commercial\Saarbrücken (1)\66125\Saarbrücker Straße 257\Commercial_Valuation_Saarbrücker Straße 257, 66125 Saarbrücken.xlsm
G:\Meine Ablage\Valuation - Commercial\Saarbrücken (1)\66125\Saarbrücker Straße 257\Einwertung NEU\Commercial_Valuation_2019_Saarbrücker Straße 257, 66125​​​​​​​ Saarbrücken.xlsm
G:\Meine Ablage\Valuation - Commercial\Saarbrücken (1)\66125\

G:\Meine Ablage\Valuation - Commercial\Saarbrücken (1)\66117 (2)\Hohenzollernstraße 124\02 Verkaufsstrategie\Commercial Valuation Hohenzollernstraße 124, 66117 Saarbrücken.xlsm
G:\Meine Ablage\Valuation - Commercial\Saarbrücken (1)\66117 (2)\Hohenzollernstraße 124\02 Verkaufsstrategie\Commercial Valuation Hohenzollernstraße 124, 66117 Saarbrücken (3).xlsm
G:\Meine Ablage\Valuation - Commercial\Saarbrücken (1)\66117 (2)\Hohenzollernstraße 124\02 Verkaufsstrategie\Commercial Valuation Hohenzollernstraße 124, 66117 Saarbrücken (2).xlsm
G:\Meine Ablage\Valuation - Commercial\Saarbrücken (1)\66117 (2)\Hohenzollernstraße 124\02 Verkaufsstrategie\Commercial Valuation Hohenzollernstraße 124, 66117 Saarbrücken (1).xlsm
G:\Meine Ablage\Valuation - Commercial\Saarbrücken (1)\66117 (2)\Reuterstr. 1\02 Verkaufsstrategie\Commercial_Valuation_Template Reuterstr. 1, 66117 Saarbrücken.xlsm
G:\Meine Ablage\Valuation - Commercial\Saarbrücken (1)\66117 (2)\Schloßstr. 48\Commercial_Valuation_2019_1_Schloßs

G:\Meine Ablage\Valuation - Commercial\Hamm\59065\Thorner Straße 6\Commercial_Valuation_2019_v5.5_DE_Thoner Straße 6, Hamm.xlsm
G:\Meine Ablage\Valuation - Commercial\Hamm\59065\Bahnhofstraße 3\Bewertung\Commercial_Valuation_2019_v5.5_DE_Bahnhofstraße 3, 59065 Hamm.xlsm
G:\Meine Ablage\Valuation - Commercial\Hamm\59065\Roonstraße 42\02 Verkaufsstrategie\Commercial Valuation Roonstraße 42, 59065 Hamm.xlsm
G:\Meine Ablage\Valuation - Commercial\Hamm\59065\Werler Straße 50\02 Verkaufsstrategie\Commercial Valuation Werler Straße 50, 59065 Hamm.xlsm
G:\Meine Ablage\Valuation - Commercial\Hamm\59065\Bockumer Weg 209\02 Verkaufsstrategie\Commercial_Valuation_Bockumer Weg 209, 59065 Hamm.xlsm
G:\Meine Ablage\Valuation - Commercial\Hamm\59065\Nordenheideweg 14\02 Verkaufsstrategie\Commercial_Valuation_Template Nordenheideweg 14, 59065 Hamm (1).xlsm
G:\Meine Ablage\Valuation - Commercial\Hamm\59065\Nordenheideweg 14\02 Verkaufsstrategie\Commercial_Valuation_Template Nordenheideweg 14, 59065 Hamm

G:\Meine Ablage\Valuation - Commercial\Potsdam\14482 (1)\Waldmüllerstr. 9\02 Verkaufsstrategie\Commercial Valuation Waldmüllerstr. 9, 14482 Potsdam.xlsm
G:\Meine Ablage\Valuation - Commercial\Potsdam\14482 (1)\Waldmüllerstr. 9\02 Verkaufsstrategie\Haupthaus\Commercial_Valuation_Template Waldmüllerstr. 9, 14482 Potsdam.xlsm
G:\Meine Ablage\Valuation - Commercial\Potsdam\14482 (1)\Waldmüllerstr. 9\02 Verkaufsstrategie\Remise\Commercial_Valuation_Template Waldmüllerstr. 9, 14482 Potsdam Souterrain.xlsm
G:\Meine Ablage\Valuation - Commercial\Potsdam\14482\Rudolf-Breitscheid-Straße 178\Bewertung\Commercial_Valuation_2019_4 Rudolf-Breitscheid-Str. 178, 14482 Potsdam.xlsm
G:\Meine Ablage\Valuation - Commercial\Potsdam\14467\Lindenstraße 46\Commercial_Valuation_2019_Lindenstraße 46, 14467 Potsdam.xlsm
G:\Meine Ablage\Valuation - Commercial\Potsdam\14473\Albert-Einstein-Straße 11A\Einwertung\Commercial_Valuation_2019_v5.5_Albert-Einstein-Straße 11A, 14473 Potsdam.xlsm
G:\Meine Ablage\Valuation 

G:\Meine Ablage\Valuation - Commercial\Ludwigshafen\67063\Gräfenaustraße 33\02 Verkaufsstrategie\Commercial_Valuation_Template Gräfenaustraße 33, 67063 Ludwigshafen.xlsm
G:\Meine Ablage\Valuation - Commercial\Ludwigshafen\67065\Maudacherstraße 157\02 Verkaufsstrategie\Commercial_Valuation_Maudacherstraße 157, 67065 Ludwigshafen.xlsm
G:\Meine Ablage\Valuation - Commercial\Ludwigshafen\67061\Rottstraße 85\02 Verkaufsstrategie\Commercial Valuation Rottstraße 85, 67061 Ludwigshafen (2).xlsm
G:\Meine Ablage\Valuation - Commercial\Ludwigshafen\67061\Rottstraße 85\02 Verkaufsstrategie\Commercial Valuation Rottstraße 85, 67061 Ludwigshafen (1).xlsm
G:\Meine Ablage\Valuation - Commercial\Ludwigshafen\67061\Rottstraße 85\02 Verkaufsstrategie\Commercial Valuation Rottstraße 85, 67061 Ludwigshafen.xlsm
G:\Meine Ablage\Valuation - Commercial\Bodman-Ludwigshafen\78351 (1)\Wiesenstraße 1\Commercial_Valuation_2019_1 Wiesenstraße 1, 78351 Bodman-Ludwigshafen.xlsm
G:\Meine Ablage\Valuation - Commercial\

G:\Meine Ablage\Valuation - Commercial\Mülheim an der Ruhr\45481 (1)\Großenbaumer Straße 51a\Einwertung\Commercial_Valuation_2019_Großenbaumer Straße 51a, 45481 Mülheim an der Ruhr.xlsm
G:\Meine Ablage\Valuation - Commercial\Mülheim an der Ruhr\45481 (1)\Am Bühl 54\Commercial_Valuation_2019_v5.5_Am Bühl 54, 45481 Mülheim an der Ruhr.xlsm
G:\Meine Ablage\Valuation - Commercial\Mülheim an der Ruhr\45481 (1)\Düsseldorfer Straße 13\Commercial_Valuation_2019_v5.5_DE Düsseldorfer Straße 13, 45481 Mülheim an der Ruhr.xlsm
G:\Meine Ablage\Valuation - Commercial\Mülheim an der Ruhr\45468\Löhberg 18-20\02 Verkaufsstrategie\Commercial Valuation Löhberg 18-20, 45468 Mülheim an der Ruhr.xlsm
G:\Meine Ablage\Valuation - Commercial\Mülheim an der Ruhr\45468\Löhberg 18-20\02 Verkaufsstrategie\Commercial_Valuation Löhberg 14, 45468 Mülheim an der Ruhr.xlsm
G:\Meine Ablage\Valuation - Commercial\Mülheim an der Ruhr\45468\Kreuzstraße 37\02 Verkaufsstrategie\Commercial Valuation Kreuzstraße 37, 45468 Mülh

G:\Meine Ablage\Valuation - Commercial\Oldenburg\26121\Deelweg 15\Commercial_Valuation_Deelweg 15.xlsm
G:\Meine Ablage\Valuation - Commercial\Oldenburg\26121\Deelweg 15\Alt\Commercial_Valuation_Deelweg 15.xlsm
G:\Meine Ablage\Valuation - Commercial\Oldenburg\26121\Johannisstraße 19\Commercial_Valuation_2019_1_Johannisstraße 19, 26121 Oldenburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Oldenburg\26121\Johannisstraße 19\02 Verkaufsstrategie\Commercial_Valuation_Template Johannisstraße 19, 26121 Oldenburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Oldenburg\26121\Fritz-Bock-Straße 5\Commercial_Valuation_2019_Fritz-Bock-Straße 5, 26121 Oldenburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Oldenburg\26121\Jägerstraße 58\Commercial_Valuation_2019_Jägerstraße 58, 26121 Oldenburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Oldenburg\26121\Melkbrink 55\Commercial_Valuation_2019_Melkbrink 55, 26121 Oldenburg, Niedersachsen.xlsm
G:\Meine Ablage\Valuation - Commercial\Oldenburg\26123\Nadorster 

G:\Meine Ablage\Valuation - Commercial\Osnabrück\49082\Spindlerstraße 27\Commercial_Valuation_2019_4_Spindelstraße 27, 49082 Osnabrück.xlsm
G:\Meine Ablage\Valuation - Commercial\Osnabrück\49082\Meller Str. 33\Verkaufsstrategie\Commercial_Valuation_2019_v5.5_Meller Str. 33, 49082 Osnabrück.xlsm
G:\Meine Ablage\Valuation - Commercial\Osnabrück\49082\Lutherstraße 18\02 Verkaufsstrategie\Commercial Valuation Lutherstraße 18, 49082 Osnabrück.xlsm
G:\Meine Ablage\Valuation - Commercial\Osnabrück\49082\Meller Straße 200\02 Verkaufsstrategie\Commercial Valuation Meller Straße 200, 49082 Osnabrück.xlsm
G:\Meine Ablage\Valuation - Commercial\Osnabrück\49076 (1)\Klosterstr. 5\Commercial_Valuation_2019_1_Klosterstr. 5, 49076 Osnabrück.xlsm
G:\Meine Ablage\Valuation - Commercial\Osnabrück\49076 (1)\Rottbrede 12\Commercial_Valuation_2019_Rottbrede 17, 49076 Osnabrück.xlsm
G:\Meine Ablage\Valuation - Commercial\Osnabrück\49076 (1)\Natruper Straße 18\Commercial_Valuation_Natruper Straße 18, 49076 Osn

G:\Meine Ablage\Valuation - Commercial\Leverkusen\51381 (1)\Burscheider Straße 476\02 Verkaufsstrategie\Commercial_Valuation_Template Burscheider Straße 476, 51381 Leverkusen.xlsm
G:\Meine Ablage\Valuation - Commercial\Leverkusen\51375\Grüner Weg 166\02 Verkaufsstrategie\Commercial Valuation Grüner Weg 166, 51375 Leverkusen.xlsm
G:\Meine Ablage\Valuation - Commercial\Leverkusen\51375\Thomas-Morus-Straße 4\Einwertung\Commercial_Valuation_Template_Thomas-Morus-Straße 4, 51375 Leverkusen.xlsm
G:\Meine Ablage\Valuation - Commercial\Leverkusen\51375\Thomas-Morus-Straße 4\Einwertung\Einzelabverkauf\Commercial_Valuation_Template_Thomas-Morus-Straße 4, 51375 Leverkusen.xlsm
G:\Meine Ablage\Valuation - Commercial\Leverkusen\51375\Thomas-Morus-Straße 4\Einwertung\MPE 2022\Commercial_Valuation_Template_Thomas-Morus-Straße 4, 51375 Leverkusen.xlsm
G:\Meine Ablage\Valuation - Commercial\Leverkusen\51375\Heinrich-Lübke-Straße 21A\02 Verkaufsstrategie\Commercial_Valuation_Template Heinrich-Lübke-Stra

G:\Meine Ablage\Valuation - Commercial\Heidelberg\69123\Im Buschgewann 34\02 Verkaufsstrategie\Commercial_Valuation_Template Im Buschgewann 34, 69123 Heidelberg.xlsm


In [55]:
top51_60     = gerCitiesSorted[50:60]
dictionary6  = dictOfroots(top51_60, subdirList)

Solingen
50
Darmstadt-West
Darmstadt
Herne-Wanne
Hernerstraße 16
Herne
50
Neuss
Neuss (1)
Regensburg
Paderborn
Ingolstadt
Offenbach an der Queich
Offenbach
Offenbach am Main
Würzburg
Fürth (Österreich)
Fürth (Odenwald)
Fürth


In [56]:
dataset6     = data_reader(dictionary6)

G:\Meine Ablage\Valuation - Commercial\Solingen\42659\Remscheider Str. 83\Commercial_Valuation_2019_1_Remscheider Str. 83 42659 Solingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Solingen\42659\Remscheider Str. 83\alt\Commercial_Valuation_2019_1_Remscheider Str. 83 42659 Solingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Solingen\42659\Grünbaumstraße 58\Commercial_Valuation_Grünbaumstraße 58, 42659 Solingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Solingen\42659\Remscheider Str. 74\Commercial_Valuation_2019_v5.5 Remscheider Str. 74, 42659 Solingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Solingen\42659\Remscheider Str. 74\02 Verkaufsstrategie\Commercial Valuation Remscheider Str. 74, 42659 Solingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Solingen\42659\Schützenstraße 19\02 Verkaufsstrategie\Commercial Valuation Schützenstraße 19, 42659 Solingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Solingen\42659\Eschbachstraße 31-35\02 Verkaufsstrategie\Commercial_Valuation_Templat

G:\Meine Ablage\Valuation - Commercial\Solingen\42719\Stresemannstraße 21\Einwertung\Commercial_Valuation_2019_v5.5_DE_Übergangsversion_Stresemannstraße 21, 42719 Solingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Solingen\42719\Bausmühlenstraße 30\02 Verkaufsstrategie\Archiv\Commercial Valuation Bausmühlenstraße 30, 42719 Solingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Solingen\42719\Stresemannstraße 33\02 Verkaufsstrategie\Commercial Valuation Stresemannstraße 33, 42719 Solingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Solingen\42719\Im Ohligs\02 Verkaufsstrategie\Commercial Valuation Im Ohligs, 42719 Solingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Solingen\42719\Sternstraße 59\02 Verkaufsstrategie\Commercial Valuation Sternstraße 59, 42719 Solingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Solingen\42719\Stresemannstraße 22\02 Verkaufsstrategie\Commercial Valuation Stresemannstraße 22, 42719 Solingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Solingen\42657\Neuenka

G:\Meine Ablage\Valuation - Commercial\Herne\44625 (1)\Bielefelder Straße 202\Verkaufsstrategie\Commercial_Valuation_2019_v5_Bielefelder Straße 202, 44625 Herne.xlsm
G:\Meine Ablage\Valuation - Commercial\Herne\44625 (1)\Dorstener Straße 75\Commercial_Valuation_2019_Dorstener Str. 75, 44625 Herne.xlsm
G:\Meine Ablage\Valuation - Commercial\Herne\44625 (1)\Dorstener Straße 75\alt\Commercial_Valuation_2019_Dorstener Str. 75, 44625 Herne.xlsm
G:\Meine Ablage\Valuation - Commercial\Herne\44625 (1)\Bielefelder Straße 86 88\02 Verkaufsstrategie\Commercial_Valuation_Bielefelder Straße 86-88, 44625 Herne.xlsm
G:\Meine Ablage\Valuation - Commercial\Herne\44625 (1)\Vödestraße 35\02 Verkaufsstrategie\Commercial Valuation Vödestraße 35, 44625 Herne.xlsm
G:\Meine Ablage\Valuation - Commercial\Herne\44625 (1)\Vödestraße 35\02 Verkaufsstrategie\Aktualisiert Commercial_Valuation_Template.xlsm
G:\Meine Ablage\Valuation - Commercial\Herne\44625 (1)\Bielefelder Straße 100 (1)\02 Verkaufsstrategie\Commerc

G:\Meine Ablage\Valuation - Commercial\Herne\44649\Karlstraße 29\02 Verkaufsstrategie\Commercial_Valuation_Template (1).xlsm
G:\Meine Ablage\Valuation - Commercial\Neuss (1)\41460\Ringbahnstraße 17\Commercial_Valuation_2019_Ringbahnstraße 17, 41460 Neuss.xlsm
G:\Meine Ablage\Valuation - Commercial\Neuss (1)\41460\Theodor-Heuss-Platz 7\Verkaufsstrategie\Commercial_Valuation_2019_v5.5_Theodor-Heuss-Platz 7, 41460 Neuss.xlsm
G:\Meine Ablage\Valuation - Commercial\Neuss (1)\41460\Breite Str. 86\Einwertung\Commercial_Valuation_2019_v5.5_DE_ Breite Str.86_41460_Neuss.xlsm
G:\Meine Ablage\Valuation - Commercial\Neuss (1)\41460\Kapitelstraße 33\Einwertung\Commercial_Valuation_2019_v5.5_DE_Kapitelstraße 33, 41460 Neuss.xlsm
G:\Meine Ablage\Valuation - Commercial\Neuss (1)\41460\Kapitelstraße 33\Einwertung neu\Commercial_Valuation_2019_v5.5_DE_Kapitelstraße 33, 41460 Neuss.xlsm
G:\Meine Ablage\Valuation - Commercial\Neuss (1)\41460\Gladbacherstraße 407 A\Commercial_Valuation_Template_Gladbacher 

G:\Meine Ablage\Valuation - Commercial\Paderborn\33098\Königsstraße 61\02 Verkaufsstrategie\Commercial_Valuation_Template Königsstraße 61, 33098 Paderborn.xlsm
G:\Meine Ablage\Valuation - Commercial\Paderborn\33098\Giselastraße 10\Commercial_Valuation_2019_Giselastraße 10, 33098 Paderborn.xlsm
G:\Meine Ablage\Valuation - Commercial\Paderborn\33098\Franziskanerstr. 2-4\Commercial_Valuation_Franziskanergasse 2-4, 33098 Paderborn.xlsm
G:\Meine Ablage\Valuation - Commercial\Paderborn\33098\Kamp 5\Commercial_Valuation_2019_v5.5_DE_Kamp 5, 33098 Paderborn.xlsm
G:\Meine Ablage\Valuation - Commercial\Paderborn\33098\Grimmestraße 20\02 Verkaufsstrategie\Commercial Valuation Grimmestraße 20, 33098 Paderborn.xlsm
G:\Meine Ablage\Valuation - Commercial\Paderborn\33098\Meinwerkstraße 3\02 Verkaufsstrategie\Commercial Valuation Meinwerkstraße 3, 33098 Paderborn.xlsm
G:\Meine Ablage\Valuation - Commercial\Paderborn\33098\Mälzerstraße 15\02 Verkaufsstrategie\Commercial Valuation Mälzerstraße 15, 33098

G:\Meine Ablage\Valuation - Commercial\Offenbach\63067\Löwenstraße 10\02 Verkaufsstrategie\Commercial Valuation Löwenstraße 10, 63067 Offenbach.xlsm
G:\Meine Ablage\Valuation - Commercial\Offenbach\63067\Löwenstraße 10\02 Verkaufsstrategie\Commercial_Valuation Löwenstraße 10, 63067 Offenbach am Main.xlsm
G:\Meine Ablage\Valuation - Commercial\Offenbach am Main\63071\63071 Offenbach, Fritz-Remy-Straße 3b - Bankrath\04_Bewertung - Verkaufsstrategie\Commercial_Valuation_Fritz-Remy-Straße 3B, 63071 Offenbach am Main.xlsm
G:\Meine Ablage\Valuation - Commercial\Offenbach am Main\63071\Dornbuschstraße 31\Einwertung\Commercial_Valuation_2019_v5.5_DE_Dornbuschstraße 31, 63071 Offenbach am Main.xlsm
G:\Meine Ablage\Valuation - Commercial\Offenbach am Main\63071\Daimlerstraße 31\02 Verkaufsstrategie\Commercial Valuation Daimlerstraße 31, 63071 Offenbach am Main.xlsm
G:\Meine Ablage\Valuation - Commercial\Offenbach am Main\63071\Grabenstraße 12\02 Verkaufsstrategie\Commercial Valuation Grabenstraß

G:\Meine Ablage\Valuation - Commercial\Würzburg\97080\Petrinistraße\02 Verkaufsstrategie\Commercial_Valuation Petrinistraße 6, 97080 Würzburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Würzburg\97082\Hartmannstraße 29\Einwertung\Commercial_Valuation_2019_v5.5_DE_Übergangsversion_Hartmannstraße 29, 97082 Würzburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Würzburg\97082\Frankfurter Straße 16\02 Verkaufsstrategie\Commercial Valuation Frankfurter Straße 16, 97082 Würzburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Würzburg\97082\Frankfurter Straße 16\02 Verkaufsstrategie\Commercial_Valuation_Frankfurter Straße 16, 97082 Würzburg_NEU.xlsm
G:\Meine Ablage\Valuation - Commercial\Würzburg\97072\Friedenstraße 29\02 Verkaufsstrategie\Commercial Valuation Friedenstraße 29, 97072 Würzburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Würzburg\97078\Untere Heerbergstraße 49\02 Verkaufsstrategie\Commercial_Valuation_Untere Heerbergstraße 49, 97078 Würzburg.xlsm
G:\Meine Ablage\Valuation - Commerci

In [57]:
top61_70     = gerCitiesSorted[60:70]
dictionary7  = dictOfroots(top61_70, subdirList)

Ulmen
Neu-Ulm
Ulm
Heilbronn
Pforzheim
Wolfsburg
Wolfsburg (1)
Göttingen
Göttingen (1)
Bottroper Straße 17
Bottrop
Bottrop (1)
Reutlingen
Koblenz
Bremerhaven
Recklinghausen
Recklinghausen (1)


In [58]:
dataset7     = data_reader(dictionary7)

G:\Meine Ablage\Valuation - Commercial\Ulmen\56766\Bahnhofstraße 43\02 Verkaufsstrategie\Commercial Valuation Bahnhofstraße 43, 56766  Ulmen.xlsm
G:\Meine Ablage\Valuation - Commercial\Neu-Ulm\89231\Memelstraße 24\Einwertung\Commercial_Valuation_2019_v5.5_DE_Memelstraße 24, 89231 Neu-Ulm.xlsm
G:\Meine Ablage\Valuation - Commercial\Neu-Ulm\89231\Bahnhofstraße 75\02 Verkaufsstrategie\Commercial_Valuation_Template Bahnhofstraße 75, 89231 Neu-Ulm.xlsm
G:\Meine Ablage\Valuation - Commercial\Neu-Ulm\89231\An der Schießmauer 15\02 Verkaufsstrategie\Commercial Valuation An der Schießmauer 15, 89231 Neu-Ulm.xlsm
G:\Meine Ablage\Valuation - Commercial\Neu-Ulm\89231\Milchweg 7\02 Verkaufsstrategie\Commercial_Valuation_Template .xlsm
G:\Meine Ablage\Valuation - Commercial\Neu-Ulm\89233\Krummenweg 17 3\Verkaufsstrategie\Commercial_Valuation_Krummenweg 17-3, 89233 Neu-Ulm, Deutschland.xlsm
G:\Meine Ablage\Valuation - Commercial\Ulm\89073\Zeitblomstraße 31\Commercial_Valuation_2019_1 Zeitblomstraße 3

G:\Meine Ablage\Valuation - Commercial\Pforzheim\75179\Kaiser-Wilhelm-Straße 7\alt\Noch älter\Commercial_Valuation_2019_v5.5_Kaiser-Wilhelm-Straße 7, 75179 Pforzheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Pforzheim\75179\Rastatter Straße 41\02 Verkaufsstrategie\Commercial_Valuation_Template Rastatter Str. 41, 75179 Pforzheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Pforzheim\75179\Mahlbergstraße 6\02 Verkaufsstrategie\Commercial Valuation Mahlbergstraße 6, 75179 Pforzheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Pforzheim\75175\Ostendstraße 3\02 Verkaufsstrategie\Commercial Valuation Ostendstraße 3, 75175 Pforzheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Pforzheim\75175\Laubgässchen 2\02 Verkaufsstrategie\Commercial Valuation Laubgässchen 2, 75175 Pforzheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Pforzheim\75175\Brühlstraße 13\Commercial_Valuation_2019_Brühlstraße 13, 75175 Pforzheim.xlsm
G:\Meine Ablage\Valuation - Commercial\Pforzheim\75175\Untere Augasse 24\Commer

G:\Meine Ablage\Valuation - Commercial\Wolfsburg (1)\38448\Helmstedter Straße 15\02 Verkaufsstrategie\Commercial Valuation Helmstedter Straße 15, 38448 Wolfsburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Göttingen (1)\37085\Beethovenstraße 14\Commercial_Valuation_2Pager_Beethovenstraße 14, 37085  Göttingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Göttingen (1)\37085\Lotzestraße 39\02 Verkaufsstrategie\Commercial Valuation  Lotzestraße 39, 37085 Göttingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Göttingen (1)\37081 (1)\Siekweg 8\Commercial_Valuation_2019_Siekweg 8, 37081 Göttingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Göttingen (1)\37081 (1)\Backhausstraße 18a\02 Verkaufsstrategie\Commercial Valuation Backhausstraße 18a, 37081 Göttingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Göttingen (1)\37081\Ebersdorfer Straße 26-28\Bewertung\Commercial_Valuation_2019_v5.5 Ebersdorfer Straße 26-28, 37081 Göttingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Göttingen (1)\37081\Eber

G:\Meine Ablage\Valuation - Commercial\Koblenz\56070 (1)\Kammertsweg 53\02 Verkaufsstrategie\Commercial Valuation Kammertsweg 53, 56070 Koblenz.xlsm
G:\Meine Ablage\Valuation - Commercial\Koblenz\56070 (1)\Kammertsweg 53\02 Verkaufsstrategie\Kammertsweg 53, 56070 Koblenz\Commercial_Valuation_Template Kammertsweg 53, 56070 Koblenz.xlsm
G:\Meine Ablage\Valuation - Commercial\Koblenz\56070 (1)\Kammertsweg 53\02 Verkaufsstrategie\Am Ufer 22, 56070 Koblenz\Commercial_Valuation_Template Am Ufer 22, 56070 Koblenz.xlsm
G:\Meine Ablage\Valuation - Commercial\Koblenz\56070 (1)\Kammertsweg 53\02 Verkaufsstrategie\Am Ufer 22, 56070 Koblenz\Scheune\Commercial_Valuation_Template Am Ufer 22, 56070 Koblenz.xlsm
G:\Meine Ablage\Valuation - Commercial\Koblenz\56070 (1)\Kammertsweg 53\02 Verkaufsstrategie\Carl-Später-Str. 75, 56070 Koblenz\Commercial_Valuation_Template Carl-Später-Str. 75, 56070 Koblenz.xlsm
G:\Meine Ablage\Valuation - Commercial\Koblenz\56070 (1)\Kammertsweg 53\02 Verkaufsstrategie\Carl

G:\Meine Ablage\Valuation - Commercial\Bremerhaven\27576\Dionysiusstr. 22\02 Verkaufsstrategie\Commercial_Valuation_Dionysiusstr. 22, 27576 Bremerhaven.xlsm
G:\Meine Ablage\Valuation - Commercial\Bremerhaven\27576\Frenssenstrasse 10\02 Verkaufsstrategie\Commercial Valuation Frenssenstrasse 10, 27576 Bremerhaven.xlsm
G:\Meine Ablage\Valuation - Commercial\Bremerhaven\27576\Lutherstraße 30\02 Verkaufsstrategie\Commercial Valuation Lutherstraße 30, 27576 Bremerhaven.xlsm
G:\Meine Ablage\Valuation - Commercial\Bremerhaven\27576\Gnesener Straße 14\02 Verkaufsstrategie\Commercial Valuation Gnesener Straße 14, 27576 Bremerhaven.xlsm
G:\Meine Ablage\Valuation - Commercial\Bremerhaven\27568\Pestalozistraße 16\02 Verkaufsstrategie\Commercial Valuation Pestalozistraße 16, 27568 Bremerhaven.xlsm
G:\Meine Ablage\Valuation - Commercial\Bremerhaven\27568\Fritz-Reuter-Straße 47+47a\02 Verkaufsstrategie\Commercial_Valuation_Template - Fritz Reuter Straße.xlsm
G:\Meine Ablage\Valuation - Commercial\Brem

G:\Meine Ablage\Valuation - Commercial\Recklinghausen (1)\45657\Immenkamp 22\02 Verkaufsstrategie\Commercial Valuation Immenkamp 22, 45657 Recklinghausen.xlsm
G:\Meine Ablage\Valuation - Commercial\Recklinghausen (1)\45661 (1)\Engelbertstraße 16\Einwertung\Commercial_Valuation_2019_v5 Engelbertstraße 16, 45661 Recklinghausen.xlsm
G:\Meine Ablage\Valuation - Commercial\Recklinghausen (1)\45661 (1)\Engelbertstraße 16\Einwertung\Alt\Commercial_Valuation_2019_v5 Engelbertstraße 16, 45661 Recklinghausen.xlsm
G:\Meine Ablage\Valuation - Commercial\Recklinghausen (1)\45661\Hochlarmarkstraße 65\Commercial_Valuation_2019_Hochlarmarkstraße 65, 45661 Recklinghausen.xlsm
G:\Meine Ablage\Valuation - Commercial\Recklinghausen (1)\45661\Bochumer Str. 104 + 106a\Einwertung\Commercial_Valuation_ÜV_Bochumer Straße 104 und 106a, 45661 Recklinghausen.xlsm
G:\Meine Ablage\Valuation - Commercial\Recklinghausen (1)\45661\Bochumer Str. 104 + 106a\Einwertung\alt\Commercial_Valuation_ÜV_Bochumer Straße 104 und 

In [4]:
top71_80     = gerCitiesSorted[70:80]
dictionary8  = dictOfroots(top71_80, subdirList)

Bergisch Gladbach
Erlangen
Jena
Remscheid
Remscheid (1)
50
Trierweiler
Trier
Salzgitter
Moers
Siegenburg
Siegen
Hildesheim
Cottbus


In [7]:
dataset8     = data_reader(dictionary8)

G:\Meine Ablage\Valuation - Commercial\Bergisch Gladbach\51467\Wagnerstraße\Commercial_Valuation_2019_4 51467 Wagnerstraße Bergisch Gladbach.xlsm
G:\Meine Ablage\Valuation - Commercial\Bergisch Gladbach\51467\Leverkusener Str. 65\Commercial_Valuation_2019_v5.5_DE_Übergangsversion_Leverkusener Str. 65, 51467 Bergisch Gladbach.xlsm
G:\Meine Ablage\Valuation - Commercial\Bergisch Gladbach\51467\Heidgen 10\Commercial_Valuation_Heidgen 10, 51467 Bergisch Gladbach.xlsm
G:\Meine Ablage\Valuation - Commercial\Bergisch Gladbach\51467\Heidgen 10\Commercial_Valuation_Heidgen 10, 51467 Bergisch Gladbach_Grundstück.xlsm
G:\Meine Ablage\Valuation - Commercial\Bergisch Gladbach\51467\Heidgen 10\alt\Commercial_Valuation_Heidgen 10, 51467 Bergisch Gladbach.xlsm
G:\Meine Ablage\Valuation - Commercial\Bergisch Gladbach\51467\Amselweg 5\02 Verkaufsstrategie\Commercial Valuation Amselweg 5, 51467 Bergisch Gladbach.xlsm
G:\Meine Ablage\Valuation - Commercial\Bergisch Gladbach\51467\Schlagbaumweg 38\02 Verka

G:\Meine Ablage\Valuation - Commercial\Jena\07743 (1)\Dornburger Straße 8\02 Verkaufsstrategie\Commercial_Valuation_Template Dornburger Straße 8, 07743 Jena.xlsm
G:\Meine Ablage\Valuation - Commercial\Jena\07743 (1)\Dornburger Straße 46 48\02 Verkaufsstrategie\Commercial_Valuation_Template Dornburger Straße 46-48, 07743 Jena.xlsm
G:\Meine Ablage\Valuation - Commercial\Jena\07743 (1)\Katharinenstraße 11\02 Verkaufsstrategie\Commercial_Valuation_Template .xlsm
G:\Meine Ablage\Valuation - Commercial\Jena\07743\Bachstraße 24\Commercial_Valuation_2019_v5.5_DE_Übergangsversion_Bachstraße 24, 07743 Jena.xlsm
G:\Meine Ablage\Valuation - Commercial\Jena\07751\Münchenroda 8\Einwertung\Commercial_Valuation_Template_Münchenroda 8, 07751 Jena.xlsm
G:\Meine Ablage\Valuation - Commercial\Jena\07749\Schenkstraße 33\02 Verkaufsstrategie\Commercial Valuation Schenkstraße 33, 07749 Jena.xlsm
G:\Meine Ablage\Valuation - Commercial\Remscheid\42855\Hägener Str. 65\02 Verkaufsstrategie\Commercial Valuation H

G:\Meine Ablage\Valuation - Commercial\Remscheid (1)\42859\Baisieper Str. 46\02 Verkaufsstrategie\Commercial Valuation Baisieper Str. 46, 42859 Remscheid.xlsm
G:\Meine Ablage\Valuation - Commercial\Remscheid (1)\42859\Hügelstrasse 25\02 Verkaufsstrategie\Commercial Valuation Hügelstrasse 25, 42859 Remscheid.xlsm
G:\Meine Ablage\Valuation - Commercial\Remscheid (1)\42859\Hügelstrasse 25\02 Verkaufsstrategie\Commercial_Valuation_Template Hügelstraße 25, 42859 Remscheid.xlsm
G:\Meine Ablage\Valuation - Commercial\Remscheid (1)\42859\Ehringhausen 63\02 Verkaufsstrategie\Commercial Valuation Ehringhausen 63, 42859 Remscheid.xlsm
G:\Meine Ablage\Valuation - Commercial\Remscheid (1)\42859\Hügelstraße 15\02 Verkaufsstrategie\Commercial Valuation Hügelstraße 15, 42859 Remscheid (1).xlsm
G:\Meine Ablage\Valuation - Commercial\Remscheid (1)\42859\Hügelstraße 15\02 Verkaufsstrategie\Commercial Valuation Hügelstraße 15, 42859 Remscheid.xlsm
G:\Meine Ablage\Valuation - Commercial\Remscheid (1)\42859

G:\Meine Ablage\Valuation - Commercial\Salzgitter\38229\Krugstrasse 2\02 Verkaufsstrategie\Commercial Valuation Krugstrasse 2, 38229 Salzgitter.xlsm
G:\Meine Ablage\Valuation - Commercial\Salzgitter\38259 (1)\Kirchpl. 2\02 Verkaufsstrategie\Commercial Valuation Kirchpl. 2, 38259 Salzgitter.xlsm
G:\Meine Ablage\Valuation - Commercial\Salzgitter\38259 (1)\Lange Str. 27\Commercial_Valuation_2019_1_Lange Str. 27, 38259 Salzgitter.xlsm
G:\Meine Ablage\Valuation - Commercial\Salzgitter\38259 (1)\Gutshof 6\Commercial_Valuation_ÜV_Gutshof 6, 38259 Salzgitter.xlsm
G:\Meine Ablage\Valuation - Commercial\Salzgitter\38259 (1)\Breite Straße 76\02 Verkaufsstrategie\Commercial_Valuation_Breite Straße 76, 38259 Salzgitter.xlsm
G:\Meine Ablage\Valuation - Commercial\Salzgitter\38259 (1)\Burgundenstraße 40\02 Verkaufsstrategie\Commercial Valuation Burgundenstraße 40, 38259 Salzgitter.xlsm
G:\Meine Ablage\Valuation - Commercial\Salzgitter\38259 (1)\Burgundenstraße 35-37\02 Verkaufsstrategie\Commercial Va

G:\Meine Ablage\Valuation - Commercial\Siegen\57078\Adolf-Wurmbach-Straße 47\02 Verkaufsstrategie\neu\Commercial_Valuation_Template_Adolf-Wurmbach-Straße 47, 57078 Siegen.xlsm
G:\Meine Ablage\Valuation - Commercial\Siegen\57072\Am Siegsteg 4\02 Verkaufsstrategie\Commercial Valuation Am Siegsteg 4, 57072 Siegen.xlsm
G:\Meine Ablage\Valuation - Commercial\Siegen\57072\Untere Metzgerstraße 13\02 Verkaufsstrategie\Commercial Valuation Untere Metzgerstraße 13, 57072 Siegen.xlsm
G:\Meine Ablage\Valuation - Commercial\Siegen\57072\Donzenbachstraße 24\02 Verkaufsstrategie\Commercial Valuation Donzenbachstraße 24, 57072 Siegen (1).xlsm
G:\Meine Ablage\Valuation - Commercial\Siegen\57072\Donzenbachstraße 24\02 Verkaufsstrategie\Commercial Valuation Donzenbachstraße 24, 57072 Siegen.xlsm
G:\Meine Ablage\Valuation - Commercial\Siegen\57072\Marburger Straße 20\02 Verkaufsstrategie\Commercial Valuation Marburger Straße 20, 57072 Siegen.xlsm
G:\Meine Ablage\Valuation - Commercial\Hildesheim\31137 (1)

In [8]:
top81_90     = gerCitiesSorted[80:90]
dictionary9  = dictOfroots(top81_90, subdirList)

Gütersloh
Kaiserslautern
Wittenburg
Wittenberge
Groß Wittensee
Witten
Lutherstadt Wittenberg
Hanau
Alt Schwerin
Schwerin
Groß-Gerau
Gerabronn
Gera
Esslingen
Esslingen am Neckar
Ludwigsburg
Iserlohn
Düren


In [9]:
dataset9     = data_reader(dictionary9)

G:\Meine Ablage\Valuation - Commercial\Gütersloh\33330 (1)\Marienfelder Str. 60\02 Verkaufsstrategie\Commercial_Valuation_Marienfelder Str.60, 33330 Gütersloh.xlsm
G:\Meine Ablage\Valuation - Commercial\Gütersloh\33330 (1)\Schillstraße 3, 33330\Commercial_Valuation_Schillstraße 3_333303 Gütersloh.xlsm
G:\Meine Ablage\Valuation - Commercial\Gütersloh\33330 (1)\Schillstraße 3, 33330\alt\Commercial_Valuation_Schillstraße3 (1).xlsm
G:\Meine Ablage\Valuation - Commercial\Gütersloh\33330 (1)\Roonstraße 17, 33330 Gütersloh\Commercial_Valuation_Roonstraße 17, 33330  Gütersloh.xlsm
G:\Meine Ablage\Valuation - Commercial\Gütersloh\33330 (1)\Heidestraße 3\Commercial_Valuation_2019_Heidestraße 3, 33330 Gütersloh.xlsm
G:\Meine Ablage\Valuation - Commercial\Gütersloh\33330 (1)\Kolpingstraße 39, 33330 Gütersloh\Commercial_Valuation_2019_v5_Kolpingstraße 39, 33330 Gütersloh.xlsm
G:\Meine Ablage\Valuation - Commercial\Gütersloh\33330 (1)\Siewekestraße 30, 33330 Gütersloh\Commercial_Valuation_2019_v5.5_

G:\Meine Ablage\Valuation - Commercial\Kaiserslautern\67657\Kennelstr. 23 u. Bierstr. 11\Verkaufsstrategie Neu\Commercial_Valuation_Kennelstr. 23 u. Bierstr. 11, 67657 Kaiserslautern.xlsm
G:\Meine Ablage\Valuation - Commercial\Kaiserslautern\67657\Friedenstraße 59\02 Verkaufsstrategie\Commercial Valuation Friedenstraße 59, 67657 Kaiserslautern.xlsm
G:\Meine Ablage\Valuation - Commercial\Kaiserslautern\67657\Friedenstraße 59\02 Verkaufsstrategie\Commercial_Valuation_Template Friedenstraße 59, 67657 Kaiserslautern.xlsm
G:\Meine Ablage\Valuation - Commercial\Kaiserslautern\67657\Ohlkasterhohl 20\02 Verkaufsstrategie\Commercial Valuation Ohlkasterhohl 20, 67657 Kaiserslautern.xlsm
G:\Meine Ablage\Valuation - Commercial\Kaiserslautern\67661\Weilerbacher Str. 63\02 Verkaufsstrategie\Commercial Valuation Weilerbacher Str. 63, 67661 Kaiserslautern.xlsm
G:\Meine Ablage\Valuation - Commercial\Kaiserslautern\67661\Eckstraße 36\02 Verkaufsstrategie\Commercial_Valuation_Template Eckstraße 36, 67661

G:\Meine Ablage\Valuation - Commercial\Lutherstadt Wittenberg\06886 (1)\Falkstraße 81\02 Verkaufsstrategie\Commercial Valuation Falkstraße 81, 06886 Lutherstadt Wittenberg.xlsm
G:\Meine Ablage\Valuation - Commercial\Lutherstadt Wittenberg\06886 (1)\Falkstraße 81\02 Verkaufsstrategie\Commercial_Valuation_Template Falkstraße 81, 06886 Lutherstadt Wittenberg.xlsm
G:\Meine Ablage\Valuation - Commercial\Lutherstadt Wittenberg\06886 (1)\Falkstraße 2\02 Verkaufsstrategie\Commercial Valuation Falkstraße 2, 06886 Lutherstadt Wittenberg.xlsm
G:\Meine Ablage\Valuation - Commercial\Lutherstadt Wittenberg\06895\Straße des Friedens 4\Commercial_Valuation_2019_v5.5 Str. des Friedens 4, 06889 Lutherstadt Wittenberg.xlsm
G:\Meine Ablage\Valuation - Commercial\Lutherstadt Wittenberg\06886\Coswiger Landstraße 65\02 Verkaufsstrategie\Commercial_Valuation_Template Coswiger Landstraße 65, 06886 Lutherstadt Wittenberg.xlsm
G:\Meine Ablage\Valuation - Commercial\Hanau\Pfaffenbrunnenstraße 129, 63456 Hanau\Com

G:\Meine Ablage\Valuation - Commercial\Gerabronn\74582\Bahnhofstraße 35\02 Verkaufsstrategie\Commercial Valuation Bahnhofstraße 35, 74582 Gerabronn.xlsm
G:\Meine Ablage\Valuation - Commercial\Gera\07546\Turmstraße 32\02 Verkaufsstrategie\Commercial Valuation Turmstraße 32, 07546 Gera.xlsm
G:\Meine Ablage\Valuation - Commercial\Gera\07546\Gerhart-Hauptmann-Str. 11\Commercial_Valuation_2019_1_Gerhart-Hauptmann-Str. 11, 07546 Gera.xlsm
G:\Meine Ablage\Valuation - Commercial\Gera\07546\Gerhart-Hauptmann-Str. 21\Commercial_Valuation_2019_1_Gerhart-Hauptmann-Str. 21, 07546 Gera.xlsm
G:\Meine Ablage\Valuation - Commercial\Gera\07546\Karl-Liebknecht-Straße 43\02 Verkaufsstrategie\Commercial_Valuation_Karl-Liebknecht-Straße 43, 07546 Gera.xlsm
G:\Meine Ablage\Valuation - Commercial\Gera\07546\Robert-Koch-Straße 7 + 23\02 Verkaufsstrategie\Commercial Valuation Robert-Koch-Straße 7 + 23, 07546 Gera.xlsm
G:\Meine Ablage\Valuation - Commercial\Gera\07546\Laasener Straße 38\02 Verkaufsstrategie\Comm

G:\Meine Ablage\Valuation - Commercial\Ludwigsburg\71634\Hartmannstraße 19\02 Verkaufsstrategie\Commercial Valuation Hartmannstraße 19, 71634 Ludwigsburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Ludwigsburg\71640\Friesenstraße 61, 71640 Ludwigsburg\Einwertung\Commercial_Valuation Friesenstraße 61, 71640 Ludwigsburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Ludwigsburg\71640\Hofweg 35\Commercial_Valuation_2019_v5.5_DE Hofweg 35, 71640 Ludwigsburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Ludwigsburg\71640\Karl-Heinrich-Käferle-Straße 1\02 Verkaufsstrategie\Commercial Valuation Karl-Heinrich-Käferle-Straße 1, 71640 Ludwigsburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Ludwigsburg\71642\Schwaikheimer Straße 16, 71642 Ludwigsburg\Einwertung\Commercial_Valuation_2019_v5.5_Schwaikheimer Straße 16, 71642 Ludwigsburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Ludwigsburg\71638\Solitudestraße 59\Commercial_Valuation_2019_Solitudestraße 59, 71638 Ludwigsburg.xlsm
G:\Meine Ablage\Valuat

G:\Meine Ablage\Valuation - Commercial\Düren\52349\Holzstraße 16\02 Verkaufsstrategie\Commercial Valuation Holzstraße 16, 52349 Düren.xlsm
G:\Meine Ablage\Valuation - Commercial\Düren\52349\Paradiesbenden 24\Verkaufsstrategie\Commercial_Valuation_ Paradiesbenden 24, 52349 Düren.xlsm
G:\Meine Ablage\Valuation - Commercial\Düren\52349\Oberstraße 18\02 Verkaufsstrategie\Commercial_Valuation_Oberstraße 18, 52349 Düren.xlsm
G:\Meine Ablage\Valuation - Commercial\Düren\52349\Josef-Schregel-Straße 76\Verkaufsstrategie\Commercial_Valuation_Template_Josef-Schregel-Straße 76, 52349 Düren.xlsm
G:\Meine Ablage\Valuation - Commercial\Düren\52349\Tivolistraße 48\02 Verkaufsstrategie\Commercial_Valuation_Template Tivolistraße 48, 52349 Düren.xlsm
G:\Meine Ablage\Valuation - Commercial\Düren\52349\Aachener Str. 57\02 Verkaufsstrategie\Commercial_Valuation_Template Aachener Str. 57, 52349 Düren.xlsm
G:\Meine Ablage\Valuation - Commercial\Düren\52351\Südstraße 22\02 Verkaufsstrategie\Commercial Valuatio

In [10]:
top91_100    = gerCitiesSorted[90:100]
dictionary10 = dictOfroots(top91_100, subdirList)

Tübingen
Flensburg
Zwickau
Gießen
Gießen (1)
Ratingen
Lünen
Villingen-Schwenningen
Konstanz
Marl
Marlow
Worms


In [11]:
dataset10    = data_reader(dictionary10)

G:\Meine Ablage\Valuation - Commercial\Tübingen\72070\Schleifmühleweg 1\Commercial_Valuation_2019_3_Schleifmühleweg 1, 72070 Tübingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Tübingen\72074\Jürgensenstraße 8\02 Verkaufsstrategie\Commercial Valuation Jürgensenstraße 8, 72074 Tübingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Tübingen\72074\Nauklerstraße 19\Commercial_Valuation_2019_v5.5­_Nauklerstraße 19, 72074 Tübingen.xlsm
G:\Meine Ablage\Valuation - Commercial\Flensburg\24943 (1)\Brixstraße 13\Commercial_Valuation_2019_1_Brixstraße 13, 24943 Flensburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Flensburg\24943 (1)\Eichenstraße 7\Commercial_Valuation_2019_v5.5_Eichenstraße 7, 24943 Flensburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Flensburg\24943 (1)\Fruerlunder Str. 50a\02 Verkaufsstrategie\Commercial Valuation Fruerlunder Str. 50a, 24943 Flensburg.xlsm
G:\Meine Ablage\Valuation - Commercial\Flensburg\24943 (1)\Bachstraße 21\02 Verkaufsstrategie\Commercial_Valuation_Template

G:\Meine Ablage\Valuation - Commercial\Gießen (1)\35390 (1)\Ederstraße 8\Alt2\Commercial_Valuation_2019_1 Ederstraße 8, 35390 Gießen.xlsm
G:\Meine Ablage\Valuation - Commercial\Gießen (1)\35390 (1)\Alicenstraße 2\02 Verkaufsstrategie\Commercial Valuation Alicenstraße 2, 35390 Gießen.xlsm
G:\Meine Ablage\Valuation - Commercial\Gießen (1)\35396\Wettenbergring 4\02 Verkaufsstrategie\Commercial Valuation Wettenbergring 4, 35396 Gießen.xlsm
G:\Meine Ablage\Valuation - Commercial\Gießen (1)\35396\Kirchstraße 26\02 Verkaufsstrategie\Commercial Valuation Kirchstraße 26, 35396 Gießen.xlsm
G:\Meine Ablage\Valuation - Commercial\Gießen (1)\35396\Kirchstraße 26\Verkaufsstrategie neu\Commercial_Valuation_Template_Kirchstraße 26, 35396 Gießen.xlsm
G:\Meine Ablage\Valuation - Commercial\Gießen (1)\35396\Gießener Str. 68\02 Verkaufsstrategie\Commercial_Valuation_Gießener Str. 68, 35396 Gießen.xlsm
G:\Meine Ablage\Valuation - Commercial\Gießen (1)\35392\Crednerstraße 36\Commercial_Valuation_2019_v5.5_D

G:\Meine Ablage\Valuation - Commercial\Konstanz\78465\Fließhornstraße 38\alt\neu\Commercial_Valuation Fließhornstrasse 38, 78465 Konstanz.xlsm
G:\Meine Ablage\Valuation - Commercial\Konstanz\78465\Fließhornstraße 38\noch älter\Commercial_Valuation_Fließhornstrasse 38, 78465 Konstanz_Kauf_Haus.xlsm
G:\Meine Ablage\Valuation - Commercial\Konstanz\78465\Fließhornstraße 38\noch älter\Commercial_Valuation_Fließhornstrasse 38, 78465 Konstanz_Residualwertverfahren.xlsm
G:\Meine Ablage\Valuation - Commercial\Konstanz\78462\Hussenstr 5, 78462 Konstanz\Einwertung - Commercial_Valuation_2019_1_Hussenstr 5, 78462 Konstanz.xlsm
G:\Meine Ablage\Valuation - Commercial\Konstanz\78462\Hussenstr 5, 78462 Konstanz\alt Verkaufsstrategie\Einwertung - Commercial_Valuation_2019_1_Hussenstr 5, 78462 Konstanz.xlsm
G:\Meine Ablage\Valuation - Commercial\Konstanz\78462\Hussenstr 5, 78462 Konstanz\Verkaufsstrategie\Commercial_Valuation_Template Hussenstr. 5, 78462 Konstanz.xlsm
G:\Meine Ablage\Valuation - Commerc